# Egyirányú n-gram-modellek

## Modell betöltése

In [1]:
%%time

import pickle

import ngram_model
from ngram_model import MultiNgramTrieRoot, BiMultiNgramModel

with open('ngram_forward.model', 'rb') as savefile:
    trie_forward = pickle.load(savefile)
with open('ngram_backward.model', 'rb') as savefile:
    trie_backward = pickle.load(savefile)

CPU times: user 49.6 s, sys: 2.47 s, total: 52 s
Wall time: 51.1 s


## A trie-csúcsokban tárolt információk

In [2]:
print(trie_forward.total)
print(trie_backward.total)

1877940021
1877940021


Ennyi karakter volt összesen a tanítókorpuszban.

In [3]:
print(trie_forward.children.keys())

dict_keys(['\x00', 'H', 'V', 'G', ' ', '9', '/', '0', '8', '.', 's', 'z', 'á', 'm', '1', 'f', 'e', 'b', 'r', 'u', '2', '7', '\n', 'A', 'ű', 'n', 'ü', 'l', 'd', 'ö', 'é', ',', 'a', 'y', 'k', 'o', 't', 'v', 'i', 'í', 'g', 'ő', 'c', 'h', 'j', 'ó', '-', 'p', 'M', '"', ':', '(', ')', 'ú', 'O', 'J', 'B', 'P', 'D', '3', 'R', 'F', 'K', 'C', 'E', 'T', 'L', '6', 'x', '5', 'I', 'N', 'S', 'Z', 'Ő', '4', 'É', 'Í', 'Á', 'U', ';', 'Ú', 'Y', 'w', 'Ü', "'", 'X', '=', '*', 'W', '?', 'Ö', 'Ó', '!', 'q', 'ä', '„', '”', 'Q', 'Ű', '&', '[', ']', 'ë', '\x01', '+', '@', '–', '%', '>', '•', '|', '$', '_'])


A tanítás közben (lásd `MultiNgramTrieRoot.main()`) a tanítósztringre meghívtam a modell példányszintű `MultiNgramTrieRoot.replace_oov()` metódusát, ami az inputsztringben kicserélte az LSTM-modell outputkódolója szerint OVV-karakternek számító karaktereket OOV-karakterekre. Ez értelemszerűen azért pont így történt, hogy közvetlenül összevethető legyen az n-gram- és az LSTM-modell, de nyilván ettől függetlenül is lehet értelme a nagyon ritka karaktereket OOV-ként kezelni. Az `\u0000` a paddingkarakter, az `\u0001` az OOV-karakter.

Hogy pontosan mi számít ritka karakternek, az a `encode_characters.py` `main()` függvényéből derül ki. Ott az `INPUT_COUNT_THRESHOLD` és a `FREQ_EXPONENT` értékét pedig úgy határoztam meg, hogy futtattam a teljes tanítókorpuszra a `press_corpus_scripts` repóból a `list_chars_in_files.py` szkriptet a korpuszra, sokáig nézegettem ennek az outputját, aztán húztam önkényesen egy határt, hogy mit különböztetek még meg külön karakterként és mi legyen már OOV.

In [4]:
print(trie_forward.children['a'].total)
print(trie_forward.prefix_count('a'))
print(trie_forward.children['a'].children['b'].total)
print(trie_forward.prefix_count('ba'))

132188621
132188621
6497044
6497044


Ennyi 'a' unigram, illetve 'ba' bigram van a korpuszban, és így tároltam a számokat.

## Simított n-gram-valószínűségek, backoff

In [5]:
trie_forward.estimate_alternatives('macsk')

{'\x00': 2.356742658931507e-08,
 'H': 2.2296633979106903e-10,
 'V': 3.4195873874692456e-10,
 'G': 5.094723100925464e-10,
 ' ': 0.003946694415040005,
 '9': 1.213029309744158e-10,
 '/': 5.531413652433361e-09,
 '0': 1.0512920684449368e-10,
 '8': 8.086862064961053e-11,
 '.': 0.0003115811380294741,
 's': 0.00010386037934315803,
 'z': 2.9130558070033216e-05,
 'á': 0.0005193018967157901,
 'm': 6.744061724394456e-07,
 '1': 2.0725472206485897e-09,
 'f': 0.00020772075868631605,
 'e': 0.008101109588766326,
 'b': 0.00020772075868631605,
 'r': 0.0006231622760589482,
 'u': 0.0021810679662063186,
 '2': 1.1067448368903842e-09,
 '7': 9.819761078881278e-11,
 '\n': 4.707015828211045e-08,
 'A': 1.3828534131083401e-09,
 'ű': 8.143123519612996e-08,
 'n': 0.004362135932412637,
 'ü': 6.922030453124063e-07,
 'l': 3.2506643915923016e-07,
 'd': 1.0713012757256691e-07,
 'ö': 0.00020772075868631605,
 'é': 0.0010386037934315801,
 ',': 0.0011424641727747382,
 'a': 0.009970596416943171,
 'y': 7.503799310077362e-08,
 

In [6]:
print(sum(trie_forward.estimate_alternatives('macsk').values()))

1.0000000000000007


Ez egy olyan dictionaryt add vissza, aminek a kulcsai a lehetséges utolsó betűk, az értékei pedig az utolsó betűknek a modell szerinti feltételes valószínűségei. Azokra az utolsó betűkre, amik a kezdő (n-1)-gram után 0 alkalommal fordultak elő, a 0 előfordulás-szám helyett a végső bigram simított valószínűségét használja előfordulásszámként (ami mindenképp kisebb, mint 0, de a végső bigramtól függ, hogy mennyivel), tehát mindenképp ritkábbnak becsülöm, mint a ténylegesen előforduló alternatíva-n-gramok darabszámát.

Például: a *macsk@* előfordulás-száma 0, így a *k@* bigram feltételes valószínűségét használom becsült előfordulásszámként.

Lehetne ugye itt is a bigram helyett az (n-1)-gramhoz backoffolni, de csak az a lényeg, hogy ez egy kellően kis szám legyen, ezért nagyjából teljesen mindegy, a bigram is megteszi.

In [7]:
print(trie_forward.prefix_count('macsk@'))
print(trie_forward.bigram_freqs['k@'])

0
2.8690037067041686e-06


Mindenesetre a tényleges előfordulásszámokat és az így becsült előfordulásszámokat összeadom az összes alternatívára, és normalizálom az egészet 1-re.

In [8]:
print(trie_forward.prefix_count('macsk'))
print(trie_forward.prefix_count('macsó'))
print(trie_forward.prefix_count('macse'))
print(trie_forward.prefix_count('macs-'))
# ...

7620
757
78
6


Backoff az n-1-gramra akkor van, ha az **összes** alternatíva előfordulásszáma 0. Például becsüljük a *macút* 5-gram valószínűségét. *macú* nincs, ezért backoffol a 4-gram-modellre. *acú* adott esetben van.

In [9]:
print(trie_forward.prefix_count('macú'))
print(trie_forward.prefix_count('acú'))

0
42


In [10]:
trie_forward.estimate_alternatives('acút')

{'\x00': 2.4285701398526387e-06,
 'H': 2.11180012161099e-08,
 'V': 1.5838500912082427e-08,
 'G': 1.055900060805495e-08,
 ' ': 0.7772882601512991,
 '9': 3.875462538059061e-12,
 '/': 2.9037251672151113e-07,
 '0': 5.279500304027475e-09,
 '8': 2.7183997592530944e-12,
 '.': 4.0187556314257146e-05,
 's': 0.023554189701554522,
 'z': 0.0008910635023131492,
 'á': 3.795960718595755e-06,
 'm': 1.3916762801416425e-05,
 '1': 5.279500304027475e-09,
 'f': 5.50704676713106e-05,
 'e': 3.880432723460194e-06,
 'b': 0.00017101885334836202,
 'r': 0.0009585196776977083,
 'u': 2.7981351611345616e-07,
 '2': 3.1677001824164854e-08,
 '7': 5.279500304027475e-09,
 '\n': 4.846581279097222e-06,
 'A': 8.44720048644396e-08,
 'ű': 6.335400364832971e-08,
 'n': 0.023554189701554522,
 'ü': 5.068320291866377e-07,
 'l': 0.004416476227829016,
 'd': 0.00020959616206989076,
 'ö': 1.8478251064096165e-07,
 'é': 5.4695623149724645e-06,
 ',': 0.09421675880621809,
 'a': 0.00015228190676936848,
 'y': 5.279500304027475e-08,
 'k': 0.

Az egész filozófiája akkor látszik jobban, ha olyan n-gramot veszünk, ahol a ténylegesen előforduló alternatívák száma nagyon kicsi:

In [11]:
print(trie_forward.prefix_count('macz'))
print(trie_forward.prefix_count('maczo'))

1
1


Az béna nyelvmodell lenne, ami ilyenkor azt mondja, hogy a *maczo* valószínűsége 1, az összes többié 0 vagy valami elenyésző kicsi szám. A használt simítási eljárással (a *za*, *zb* stb. bigramok valószínűségével becsüljük 0 előfordulású 5-gramok előfordulás-számát) normalizálás után sokkal értelmesebb számokat kapunk:

In [12]:
trie_forward.estimate_alternatives('maczó')

{'\x00': 5.229501968233869e-05,
 'H': 1.4106556369031632e-06,
 'V': 7.983380802254165e-07,
 'G': 8.060889353732362e-07,
 ' ': 0.09028946358958553,
 '9': 6.200684118255663e-08,
 '/': 4.40248572396152e-06,
 '0': 2.4027650958240696e-07,
 '8': 3.100342059127832e-07,
 '.': 0.0018609570684260375,
 's': 0.00926641860914848,
 'z': 0.007809412858461355,
 'á': 0.041357098157142336,
 'm': 0.0030530695935812794,
 '1': 9.223517625905299e-07,
 'f': 0.0005735090249526141,
 'e': 0.09277016353392092,
 'b': 0.0033183658635808485,
 'r': 0.0018402080291953244,
 'u': 0.003134639593156933,
 '2': 2.7825569980672285e-06,
 '7': 1.47266247808572e-07,
 '\n': 0.00010470630219189469,
 'A': 2.146986875946023e-06,
 'ű': 0.0007967104006443744,
 'n': 0.009385122955737337,
 'ü': 0.008808583346421926,
 'l': 0.006233943037694956,
 'd': 0.007611293250027939,
 'ö': 0.011505036094652027,
 'é': 0.022048392587693484,
 ',': 0.0038965951593184847,
 'a': 0.03824595915679359,
 'y': 0.00032281536605153767,
 'k': 0.0042303857362593

Csak összehasonlításképp így nézne ki (a *maczo* 1-ének hozzáadása a normalizálás előtt), ha nem azonnal a bigram-modellre, hanem a 4-gram-modellre lépnénk vissza:

In [13]:
alts = trie_forward.estimate_alternatives('aczó')
# maczo-ból 1 van
alts['o'] = 1
# normalizálás
counts_sum = sum(alts.values())
alts = {k: v / counts_sum for k, v in alts.items()}

In [14]:
alts

{'\x00': 9.150216774292263e-09,
 'H': 2.468266567246468e-10,
 'V': 1.3968761342109133e-10,
 'G': 1.4104380384265534e-10,
 ' ': 0.020015446009443764,
 '9': 9.015966670920614e-05,
 '/': 7.703161594483483e-10,
 '0': 4.20419030684838e-11,
 '8': 5.424761686255975e-11,
 '.': 0.00018031933341841228,
 's': 0.00027047900012761847,
 'z': 1.3664364401989094e-06,
 'á': 0.004147344668623483,
 'm': 0.008294689337246966,
 '1': 1.6138666016611522e-10,
 'f': 0.00018031933341841228,
 'e': 0.00703245400331808,
 'b': 0.00027047900012761847,
 'r': 3.219867298877234e-07,
 'u': 0.002434311001148566,
 '2': 4.868723613414737e-10,
 '7': 2.5767618009715876e-11,
 '\n': 1.832077640490799e-08,
 'A': 3.7566474677322617e-10,
 'ű': 1.394028134325629e-07,
 'n': 0.0005409580002552369,
 'ü': 1.5412643036331454e-06,
 'l': 1.090771750350126e-06,
 'd': 1.3317708568333121e-06,
 'ö': 2.0130707455814647e-06,
 'é': 0.00351622700165904,
 ',': 0.002253991667730153,
 'a': 0.012712513005998067,
 'y': 0.15615654274034504,
 'k': 0.14

Úgy voltam vele, hogy miután ezt a rekurzív struktúrában sokkal költségesebb lenne mindig kiszámolni, másrészt annyira drasztikusan nem tér el a bigramos közelítéstől (bár az utóbbi adott esetben persze csúnyán alábecsüli a *czy* és a *czk* valószínűségét), nem éri meg belefektetni a plusz számolási munkát és az algoritmus bonyolítását.

Mindenesetre visszatérve a backoffra: ha a kezdő 3-gram sem fordult elő, visszalépünk a trigram-modellre, ha pedig az sem létezne, akkor a bigram-modellre. Becsüljük a *zxűxt* 5-gram valószínűségét:

In [15]:
print(trie_forward.prefix_count('zxűx'))
print(trie_forward.prefix_count('xűx'))
print(trie_forward.prefix_count('űx'))
print(trie_forward.bigram_freqs['xt'])

0
0
0
0.05202814798238691


A teljes bigram-modell (tehát az ábécében szereplő összes karakter kombinációja) már eleve simítva készen el van tárolva, hogy ennyivel gyorsabban fusson a modell. (Az elő nem forduló bigramoknál simán összeszoroztam a két unigram valószínűségét.)

In [16]:
print(len(trie_forward.children.keys()))
print(len(trie_forward.children.keys()) ** 2)
print(len(trie_forward.bigram_freqs.keys()))

114
12996
12996


A 114 az az LSTM-beli 113 outputkarakter plusz a padding. Az LSTM soha nem jelez előre paddinget a tanítás menete miatt, hanem az csak kontextusként funkcionál, az n-gram-modellben viszont értelemszerűen megegyezik az output (utolsó karakter) és az input (a feltétel) ábécéje, így ezzel az eggyel több karakter van az ábécéjében. Kis szépséghiba, hogy a betanításkor paddingre végződő n-gramokra is tanítottam a modellt (nem csak a vele kezdődőkre), tehát ebben eltér az n-gram-modell tanítása az LSTM-modellétől, de nagyjából teljesen mindegy.

## Kényelmi metódusok

Az `estimate_prob()` az n-gram utolsó karakterének becsült (lényegében maximum likelihood-becslés alapján + a 0 előfordulású alternatívák simítása) feltételes valószínűségét adja vissza adott n-1 hosszú prefixum után.

In [17]:
trie_forward.estimate_prob('macsk')

0.7914160905948642

Ugyanaz, mint:

In [18]:
trie_forward.estimate_alternatives('macsk')['k']

0.7914160905948642

Az `estimate_alternatives()` az ábécé szerint lehetséges összes utolsó karakter feltételes valószínűségét adja vissza dictionary formájában.

Ha túl hosszú n-gramot adunk meg, akkor az n hosszúságú szuffixuma alapján becsül.

In [19]:
print(trie_forward.estimate_prob('macskákat', verbose=True))
print(trie_forward.estimate_prob('kákat'))

macskákat is longer than the model's order, estimating for acskákat
macskáka not seen in corpus, estimating based on acskáka
acskákat is longer than the model's order, estimating for cskákat
acskáka not seen in corpus, estimating based on cskáka
cskákat is longer than the model's order, estimating for skákat
cskáka not seen in corpus, estimating based on skáka
skákat is longer than the model's order, estimating for kákat
0.9978041429885643
0.9978041429885643


In [20]:
trie_forward.estimate_alternatives('macskákat')['t']

0.9978041429885643

Ha 0 gyakoriságú a feltétel (tehát az összes alternatív 5-gram gyakorisága 0 a tanítókorpuszban), akkor backoffol, amíg nem jut el egy nem 0 gyakoriságúig vagy a bigramig:

In [21]:
print(trie_forward.estimate_prob('zxűxt', verbose=True))
print(trie_forward.estimate_prob('xt', verbose=True))
print(trie_forward.estimate_alternatives('zxűxt')['t'])

zxűx not seen in corpus, estimating based on xűx
xűx not seen in corpus, estimating based on űx
űx not seen in corpus, estimating based on x
0.05202814798238691
0.05202814798238691
0.05202814798238689


Ha n-nél rövidebb sztringet adunk meg, akkor szintén backoffol impliciten a megfelelő rendű n-gram-modellre:

In [22]:
print(trie_forward.estimate_prob('cska', verbose=True))
print(trie_forward.estimate_alternatives('cska')['a'])

0.1979940120293708
0.1979940120293708


In [23]:
print(trie_forward.estimate_prob('ska', verbose=True))
print(trie_forward.estimate_alternatives('ska')['a'])

0.07334865741519339
0.07334865741519339


A `predict_next()` a megadott sztring **után következő** lehetséges karakterek közül csökkenő sorrendben az m legvalószínűbbet adja vissza listaként.

In [24]:
print(trie_forward.predict_next('macs'))
print(trie_forward.predict_next('macs', 4))

['k']
['k', 'ó', 'ú', 'o']


A `metrics on string()` végigléptet egy n hosszúságú ablakot az inputsztringen, és kiszámítja a sztringre a karakterenkénti perplexitást és a `predict_next()` által jósolt legvalószínűbb karakter pontosságát (tehát ténylegesen az-e a következő karakter vagy nem).

Opcionális egy-egy paddingkaraktert is ad hozzá az elejéhez és a végéhez, ebben az esetben az első karakter valószínűségét a padding után a bigram-modell alapján számolja, aztán a 2. karakter valószínűségét a padding és az 1. után a trigram-modell alapján stb., majd a sztring végéig az összes teljes n-gramét, ami adott esetben 5-gram.

Ha nincs padding, akkor csak a teljes 5-gramokat számolja, tehát az első 4 karakterre nem számít becsült valószínűséget és jósolt karaktert.

In [1]:
long_string_1 = '''A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.'''
long_string_2 = '''Tavaly októberben adtunk hírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Joshi Bharattal alkot egy párt. Most viszont a Story alig pár hónappal később arról számolt be, hogy véget ért kettejük kapcsolata, a lap információi szerint Joshi Bharat már el is költözött abból a budai villából, ahol eddig együtt éltek.
Kelemen Beatrix hosszú évekig élt házasságban Mészáros Lőrinccel. A pár 2020 őszén adta be a váló pert, a részletekről, így a vagyonról is hamar megegyeztek. Egy évvel később már arról lehetett hallani, hogy Kelemen Beatrix Joshi Bharattal alkot egy párt.
Közben pedig, ki ne tudná, Mészáros Lőrinc is újra szerelmes lett és már újra is nősült. Az ország egyik leggazdagabb embere a TV2 korábbi műsorvezetőjével, Várkonyi Andreával kötötte össze az életét.'''
print(long_string_1)
print()
print(long_string_2)

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.

Tavaly októberben adtunk hírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Joshi Bhara

In [26]:
%%time
print(len(long_string_1))
print(trie_forward.metrics_on_string(long_string_1))
print(trie_forward.metrics_on_string(long_string_1, padded=True))

858
(0.5831381733021077, 3.744253215919672)
(0.5820721769499418, 3.7884149831881415)
CPU times: user 628 ms, sys: 8.22 ms, total: 636 ms
Wall time: 634 ms


In [27]:
%%time
print(len(long_string_2))
print(trie_forward.metrics_on_string(long_string_2))
print(trie_forward.metrics_on_string(long_string_2, padded=True))

825
(0.5152253349573691, 4.328136190221757)
(0.5145278450363197, 4.335518273647436)
CPU times: user 610 ms, sys: 3.73 ms, total: 614 ms
Wall time: 612 ms


Lehet a következő karakter előrejelzése nélkül csak a perplexitást is számítani. Ez értelemszerűen minimálisan gyorsabb, mindenesetre az OCR-javításhoz, szemétszűréshez, nyelvfelismeréshez és hasonlókhoz csak erre van szükség. Ez a függvény a paddinggel sem szórakozik, mert az általában nem sokat számít, tehát ha pl. OCR-javításra akarjuk használni, akkor a gyanús részsztringet balra legalább n-1 karakter ablakkal célszerű átadni neki.

In [28]:
%%time
print(trie_forward.metrics_on_string(long_string_1))
print(trie_forward.metrics_on_string(long_string_2))

(0.5831381733021077, 3.744253215919672)
(0.5152253349573691, 4.328136190221757)
CPU times: user 628 ms, sys: 23 µs, total: 628 ms
Wall time: 627 ms


In [29]:
%%time
print(trie_forward.string_perplexity(long_string_1))
print(trie_forward.string_perplexity(long_string_2))

3.744253215919672
4.328136190221757
CPU times: user 600 ms, sys: 8 µs, total: 600 ms
Wall time: 598 ms


Ha ezt profilozzuk, jól látszik, hogy a futási idő messze legnagyobb részét a `prefix_count()` rekurzív hívásai teszik ki.

Egyébként a jelenlegi már egy teljesítményre optimalizált kód. Amikor első körben megírtam, akkor még vagy ötvenszer ennyi idő alatt futott le, ami használhatatlan volt gyakorlatilag. Sokáig gondolkodtam rajta, hogy hol végez felesleges munkát, és így jutottam el a jelenlegi változathoz.

In [30]:
import cProfile
cProfile.run('trie_forward.metrics_on_string(long_string_1)')

         991967 function calls (650567 primitive calls) in 0.443 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.443    0.443 <string>:1(<module>)
      855    0.001    0.000    0.001    0.000 more.py:832(windowed)
      854    0.055    0.000    0.410    0.000 ngram_model.py:181(estimate_alternatives)
      854    0.009    0.000    0.009    0.000 ngram_model.py:219(<dictcomp>)
      854    0.001    0.000    0.414    0.000 ngram_model.py:249(backoff_alternatives)
        1    0.000    0.000    0.001    0.001 ngram_model.py:288(replace_oov)
        1    0.003    0.003    0.443    0.443 ngram_model.py:303(metrics_on_string)
    97356    0.007    0.000    0.007    0.000 ngram_model.py:347(<lambda>)
439610/98210    0.325    0.000    0.347    0.000 ngram_model.py:98(prefix_count)
        1    0.000    0.000    0.443    0.443 {built-in method builtins.exec}
   443028    0.022    0.000    0.022    0

## Jobbról balra haladó n-gram-modell

A modell megfordított sztringeken lett betanítva, ezért az n-gramokat is megfordítva kell neki odaadni inputként.

In [31]:
trie_backward.estimate_prob('macsk'[::-1])

0.4487024414014663

In [32]:
trie_backward.estimate_alternatives('macsk'[::-1])['m']

0.4487024414014663

In [33]:
trie_backward.predict_next('acsk'[::-1], 5)

['m', 'z', 'M', 'h', 's']

Az előre és a hátra modell perplexitása majdnem azonos mindkét sztringen.

In [34]:
print(trie_forward.metrics_on_string(long_string_1))
print(trie_backward.metrics_on_string(long_string_1[::-1]))

(0.5831381733021077, 3.744253215919672)
(0.5737704918032787, 3.784722241197091)


In [35]:
print(trie_forward.metrics_on_string(long_string_2))
print(trie_backward.metrics_on_string(long_string_2[::-1]))

(0.5152253349573691, 4.328136190221757)
(0.5505481120584653, 4.314408348738489)


Ha elfelejtjük megfordítani az inputot, az azonnal látszik, mert borzalmasan rossz számokat kapunk.

In [36]:
print(trie_backward.metrics_on_string(long_string_2))

(0.0779537149817296, 720.5035030526914)


# Kétirányú n-gram-modell

Két egyirányúból példányosítjuk. Az előre és a hátra modellnek azonos ábécén kell alapulnia, más követelmény nincs, így például konkrétan nem kell azonos rendűnek sem lenniük, lehet pl. előre 5-gram-, hátra 3-gram-modell.

In [37]:
bi_ngram_model = BiMultiNgramModel(trie_forward, trie_backward)

A két modell összekapcsolását a `combine_alternatives()` metódus végzi, ami az egyirányú modellbeli `estimate_alternatives()` megfelelője, és ahhoz hasonlóan működik. Külön megadunk neki egy előre és egy hátra n-gramot (az utóbbit az átadáskor meg kell fordítani), ezeket backoff stb. szempontjából külön-külön feldolgozza a megfelelő egyirányú modell, majd az ezektől kapott két valószínűségi eloszlást a kulcsok (tehát az n-gramok utolsó eleme) szerint páronként összekapcsolja a választott összekapcsoló függvénnyel, majd az eredményt normalizálja valószínűségi eloszlásra. Ha nem adunk meg összekapcsoló függvényt, az alapértelmezés szerint `sum` függvény lesz, tehát az előre és a hátra modell által az egyes karakterekre becsült valószínűségeket átlagoljuk.

In [38]:
bi_ngram_model.combine_alternatives('lakót', 'telep'[::-1])

{'\x00': 0.00013767875595448775,
 'H': 4.242259312272742e-09,
 'V': 8.180856199347752e-06,
 'G': 3.1244289203836156e-09,
 ' ': 0.045867380011428746,
 '9': 1.0408610633787692e-11,
 '/': 9.834252525464875e-06,
 '0': 8.413437666872677e-12,
 '8': 5.3124917666970005e-12,
 '.': 0.0013120381343120753,
 's': 0.0023898830032439175,
 'z': 0.008641838005827941,
 'á': 7.097315442389541e-09,
 'm': 0.00031304039378733244,
 '1': 8.157153039521859e-11,
 'f': 0.00019679037719343346,
 'e': 0.0011014337514695315,
 'b': 0.0047348888910285535,
 'r': 0.0015857417472511215,
 'u': 6.884246211834925e-05,
 '2': 6.960333362839322e-11,
 '7': 5.5179886958907124e-11,
 '\n': 0.00032452848799666443,
 'A': 2.1677185107103502e-10,
 'ű': 3.024826283770442e-09,
 'n': 0.008074118786932432,
 'ü': 9.834232409748387e-06,
 'l': 0.019906080310504202,
 'd': 4.9315066003478194e-05,
 'ö': 0.002104517801375226,
 'é': 0.013266329452048958,
 ',': 0.004553232383189455,
 'a': 0.001986512507355914,
 'y': 1.5088121130771684e-07,
 'k': 0

Mivel a két n-gram összes lehetséges alternatívájára számítjuk a valószínűségeket, ezért az előre n-gram utolsó és a hátra n-gram első karakterére teljesen mindegy, hogy mit írunk be, ugyanazt a valószínűségi eloszlást kapjuk vissza.

In [39]:
bi_ngram_model.combine_alternatives('lakóp', 'zelep'[::-1])

{'\x00': 0.00013767875595448775,
 'H': 4.242259312272742e-09,
 'V': 8.180856199347752e-06,
 'G': 3.1244289203836156e-09,
 ' ': 0.045867380011428746,
 '9': 1.0408610633787692e-11,
 '/': 9.834252525464875e-06,
 '0': 8.413437666872677e-12,
 '8': 5.3124917666970005e-12,
 '.': 0.0013120381343120753,
 's': 0.0023898830032439175,
 'z': 0.008641838005827941,
 'á': 7.097315442389541e-09,
 'm': 0.00031304039378733244,
 '1': 8.157153039521859e-11,
 'f': 0.00019679037719343346,
 'e': 0.0011014337514695315,
 'b': 0.0047348888910285535,
 'r': 0.0015857417472511215,
 'u': 6.884246211834925e-05,
 '2': 6.960333362839322e-11,
 '7': 5.5179886958907124e-11,
 '\n': 0.00032452848799666443,
 'A': 2.1677185107103502e-10,
 'ű': 3.024826283770442e-09,
 'n': 0.008074118786932432,
 'ü': 9.834232409748387e-06,
 'l': 0.019906080310504202,
 'd': 4.9315066003478194e-05,
 'ö': 0.002104517801375226,
 'é': 0.013266329452048958,
 ',': 0.004553232383189455,
 'a': 0.001986512507355914,
 'y': 1.5088121130771684e-07,
 'k': 0

Tetszőleges függvényt megadhatunk, ami kételemű listából értelmezhető numerikus eredményt ad, pl. össze is szorozhatjuk a két külön becsült valószínűséget, vagy számolhatjuk a harmonikus közepüket.

In [40]:
multiply = lambda x: x[0] * x[1]
bi_ngram_model.combine_alternatives('lakóp', 'zelep'[::-1], combine_function=multiply)

{'\x00': 1.337648372080904e-13,
 'H': 4.763290958722882e-18,
 'V': 1.7772330328685344e-13,
 'G': 2.6308211943262398e-18,
 ' ': 0.00013460366280637813,
 '9': 1.139846206354772e-21,
 '/': 2.382409237517969e-14,
 '0': 7.299507725424648e-22,
 '8': 2.5688652187552125e-22,
 '.': 2.2612748870452182e-07,
 's': 1.753206000848735e-08,
 'z': 2.4945432438140932e-05,
 'á': 1.930312705443978e-17,
 'm': 1.0541281428331091e-07,
 '1': 1.0593410586522519e-20,
 'f': 8.851758018383823e-10,
 'e': 1.8099766248432653e-10,
 'b': 0.00015362387495778778,
 'r': 8.160992073546652e-07,
 'u': 9.007574626359783e-12,
 '2': 4.480774742222206e-21,
 '7': 5.190371118222622e-21,
 '\n': 6.309805753100165e-13,
 'A': 4.948013424223668e-19,
 'ű': 1.7783608183758933e-18,
 'n': 8.344761283819867e-08,
 'ü': 1.5462369920181e-14,
 'l': 1.653280977565993e-05,
 'd': 2.9947730512778723e-10,
 'ö': 2.3121298691665802e-12,
 'é': 6.222371220472896e-11,
 ',': 6.164977953717139e-17,
 'a': 4.260949259843597e-10,
 'y': 5.576327872599458e-16,

In [41]:
harmonic_mean = lambda x: 2 / (1 / x[0] + 1 / x[1])
bi_ngram_model.combine_alternatives('lakóp', 'zelep'[::-1], combine_function=harmonic_mean)

{'\x00': 4.791232883040705e-10,
 'H': 5.537097200823365e-10,
 'V': 1.0713166790851864e-08,
 'G': 4.1523402329463514e-10,
 ' ': 0.0014471888004790763,
 '9': 5.400397703609621e-11,
 '/': 1.1946680711752337e-09,
 '0': 4.278507912901896e-11,
 '8': 2.3845986685818305e-11,
 '.': 8.499221549475826e-05,
 's': 3.6176667341664355e-06,
 'z': 0.0014234989935525758,
 'á': 1.341237877518072e-09,
 'm': 0.00016606009247757954,
 '1': 6.404267201245744e-11,
 'f': 2.21818581337544e-06,
 'e': 8.103764918485307e-08,
 'b': 0.016000043802501005,
 'r': 0.00025379481465717755,
 'u': 6.452436260046918e-08,
 '2': 3.174646107970814e-11,
 '7': 4.638631294294664e-11,
 '\n': 9.588163074259369e-10,
 'A': 1.125641850685129e-09,
 'ű': 2.8992902269799735e-10,
 'n': 5.09672300637434e-06,
 'ü': 7.753679402490972e-10,
 'l': 0.0004095747411375496,
 'd': 2.9947222286457543e-06,
 'ö': 5.417911029963247e-10,
 'é': 2.3130093448300056e-09,
 ',': 6.67703973024538e-15,
 'a': 1.0577604078805957e-07,
 'y': 1.822574911015894e-09,
 'k

A kényelmi metódusok lényegében megegyeznek az egyirányú modellben rendelkezésre állókkal, de a paramétereik eltérnek annak következtében, hogy kétirányú a modell.

A `predict_next()` metódusnak a keresett középső elemtől balra és jobbra eső (n-1)-gramot kell átadni, és az utóbbit meg kell fordítani.

In [42]:
print(bi_ngram_model.predict_next('lakó', 'elep'[::-1]))
print(bi_ngram_model.predict_next('egol', 'ást '[::-1], m=4))

['t']
['d', 't', 'l', 'z']


Az `estimate_prob()` metódus ugyanúgy képes kezelni olyan helyzeteket, amikor rövidebb vagy hosszabb bal vagy jobb n-gramot adunk át neki a két egyirányú modell rendjénél, mint ahogy a metódus egyirányú megfelelője is tudja ezeket kezelni, viszont ahhoz, hogy tudja, hogy melyik karakter valószínűségét kell megbecsülnie, ilyen helyzetekben meg kell adni a kérdéses elem indexét második vagy nevesített argumentumként.

Ha ezt nem tesszük meg, alapértelmezés szerint az n-edik elem valószínűségét becsüli meg, ahol n az előre modell rendje.

In [43]:
print(len('egoldást '))
print(bi_ngram_model.estimate_prob('egoldást '))
print(bi_ngram_model.estimate_prob('egoldást ', 4)) # d
print(bi_ngram_model.estimate_prob('egoldást ', 3)) # l
print(bi_ngram_model.estimate_prob('egoldást ', 2)) # o

print(bi_ngram_model.estimate_prob('macskát', 3)) # s
print(bi_ngram_model.estimate_prob('macskát')) # k

9
0.5419201827236841
0.5419201827236841
0.45670667893237255
0.47380608207172187
0.14087125462165861
0.4133456382486001


A `metrics_on_string()` metódus hasonlóan egy ablakot léptet végig a sztringet, és lépésenként az előre és hátra n-gram metszéspontjában levő karaktert értékeli ki.

Ha van padding, akkor nemcsak az elején értékeli ki az első karaktert a padding utáni bigram, a másodikat a trigram stb. alapján, hanem ugyanezt csinálja fordítva a sztring végén is. Ha nincs padding, akkor simán teljes ablakokat léptet.

In [44]:
%%time
print(trie_forward.metrics_on_string(long_string_1))
print(trie_backward.metrics_on_string(long_string_1[::-1]))

(0.5831381733021077, 3.744253215919672)
(0.5737704918032787, 3.784722241197091)
CPU times: user 636 ms, sys: 11.6 ms, total: 647 ms
Wall time: 646 ms


In [45]:
%%time
print(bi_ngram_model.metrics_on_string(long_string_2))

(0.7356181150550796, 2.982173827104181)
CPU times: user 699 ms, sys: 0 ns, total: 699 ms
Wall time: 698 ms


Perplexitások különböző összekapcsoló függvényekre. Úgy tűnik, ezek közül következetesen a **szorzás** adja a legjobb eredményeket mind pontosságra, mind perplexitásra nézve, illetve a pontossága nyilván megegyezik a mértani középével. A tanulság az, hogy tényleg érdemes különböző összekapcsoló függvényekkel kísérletezni.

In [46]:
geometric_mean = lambda x: (x[0] * x[1]) ** 2
max_squared = lambda x: max(x) ** 2

In [47]:
print(bi_ngram_model.metrics_on_string(long_string_1)) # sum
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=max))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=max_squared))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=geometric_mean))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=harmonic_mean))

(0.8058823529411765, 2.630060632382658)
(0.7705882352941177, 2.928863960323583)
(0.7705882352941177, 2.1839104337061452)
(0.8623529411764705, 1.5094330610760718)
(0.8623529411764705, 1.7145876208025843)
(0.8247058823529412, 1.7446173300756533)


In [48]:
print(bi_ngram_model.metrics_on_string(long_string_2)) # sum
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=max))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=max_squared))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=geometric_mean))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=harmonic_mean))

(0.7356181150550796, 2.982173827104181)
(0.6988984088127295, 3.322779019968607)
(0.6988984088127295, 2.609475595243155)
(0.7968176254589964, 1.7982801627641019)
(0.7968176254589964, 2.273808259070984)
(0.7723378212974297, 2.052976764170971)


Azt is kiírathatjuk, hogy az egész sztringben melyik karaktereket jelzi rendre előre legvalószínűbbként a modell:

In [49]:
print(long_string_1)

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.


In [50]:
bi_ngram_model.metrics_on_string(long_string_1, print_string=True, padded=True)

A  talthatatlan a puslolósa helyeet a lakoteletetet  című cikkünk en részzeteten bemutatjak, hogy a Budapest sűrűs lakost várásrészetben eső autósok számára egyre többször ohaz lllm  problémák a purkol hály keresig. A folytatásban UUh méter Tamás m levetségit megoldások t mutat a ae. összefoglalka  hi válható a jövőben  ellette aátte egyáltat l a purkol mely a  egy e szaborodá aatók számára a ks kereletebet  vagy laosan vel kell hagytink a  eddig  életmudunk al. némány lele részzetet ciekeen foglalkoztank a  Ózudan kialakult puslolósa khrszzat, m után kikerült, hogy más a helye ks kerelet feligyelet nem minden szabolytttan a tót bentet megt ennyite e fogaai  a helyet. A cikk megjelenéret követben zarorodaat a  olvashi lehenett és  hogy válható volt   endedeget szerették, ho a  erent tt hivatalos végte megoldást taráltinak a súlyoskídi problémál k


(0.8018648018648019, 2.7088778690523965)

In [51]:
bi_ngram_model.metrics_on_string(long_string_1, combine_function=multiply, print_string=True, padded=True)

A ktarthatatlan a partolása helyzet a lakiteletetet, című cikkünkben részletesen bemutatjak, hogy a budapest serűs lakost városrészetben eső autósok számára egyre többször okaz ellmb problémák a parkolókály keretég. A folytatásban ajh Péter Tamás m levetséget megoldásokat mutat a be. Összefoglalta, aa válható a jövőben, ellette aátte egyáltatál a parkolókely az egyse szaborodt aztók számára a köztereletetet, vagy lassan vel kell hagytink az eddigi lletlódunkkal. Némány keli részletet cikkben foglalkottank az ezudan kialakula partolása kroszzat, miután kikerült, hogy más a helye közterelet felegyeket nem minden szabolytatan aztót tentet megt annyita elfogltik a helyet. A ciák megjelenéset követsen zasorodnat az olvasni lesenett és ahogy válható volta hengedegek szerették, ha az erintott hivatalos végre megoldást tanáltinak a súlyossodi problémál K


(0.8578088578088578, 1.5973799384302159)

## OCR-hibák, nem magyar szöveg

In [13]:
string1_correct = "A Matáv Adományvonal lehetővé teszi, hogy ön egyszerűen és gyorsan juttasson célba adományt, míg a nonprofit szervezetek pályázati úton, rövid idő alatt juthatnak nagyobb összegekhez egészségmegőrző, szociális es gyógyító programjaikhoz."
string1_ocr = "A MatPv AdomPny vonal /hl^'etVvö tesiri, hgyy ön egyszerűen és gyorsan UittUssson cáia adományt, míg a non-profit szervezetek pályázati úton, dVvdJ kföalatt uutaatnak nagobür összegekhez ^fS^^e^í^zö, tooPPHs és gyógyító'programjaihhzz."
print(string1_correct)
print(string1_ocr)

A Matáv Adományvonal lehetővé teszi, hogy ön egyszerűen és gyorsan juttasson célba adományt, míg a nonprofit szervezetek pályázati úton, rövid idő alatt juthatnak nagyobb összegekhez egészségmegőrző, szociális es gyógyító programjaikhoz.
A MatPv AdomPny vonal /hl^'etVvö tesiri, hgyy ön egyszerűen és gyorsan UittUssson cáia adományt, míg a non-profit szervezetek pályázati úton, dVvdJ kföalatt uutaatnak nagobür összegekhez ^fS^^e^í^zö, tooPPHs és gyógyító'programjaihhzz.


In [53]:
print(bi_ngram_model.metrics_on_string(string1_correct))
print(bi_ngram_model.metrics_on_string(string1_ocr))

(0.8209606986899564, 2.564246492735871)
(0.46255506607929514, 25.62730166071084)


Összeadással is szépen különválik a jó és az OCR-hibás szöveg perplexitása, de szorzással még jobban:

In [54]:
print(bi_ngram_model.metrics_on_string(string1_correct, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string1_ocr, combine_function=multiply))

(0.8384279475982532, 1.5190116169969372)
(0.5022026431718062, 319.6578162054826)


In [14]:
string2_good_ocr = "Környezetvédelem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól.	a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése)	".replace('\t', ' ')
string2_bad_ocr = "K^t^r^yt^zz^t^i^tédielem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du-	- A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól.	a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése)	".replace('\t', ' ')
print(string2_good_ocr)
print(string2_bad_ocr)

Környezetvédelem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól. a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése) 
K^t^r^yt^zz^t^i^tédielem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól. a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése) 


In [56]:
print(bi_ngram_model.metrics_on_string(string2_good_ocr))
print(bi_ngram_model.metrics_on_string(string2_bad_ocr))

(0.7605633802816901, 3.1477087801617234)
(0.6877828054298643, 5.251298562043095)


In [57]:
print(bi_ngram_model.metrics_on_string(string2_good_ocr, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string2_bad_ocr, combine_function=multiply))

(0.8169014084507042, 2.148832287267806)
(0.746606334841629, 6.035245541728297)


Az alábbiak azok a rövid OCR-hibás sztringek, amikre panaszkodtam a 15-ös issue-ban, hogy a nyelvfelismerők kiváló magyar szövegként azonosítják őket, illetve még rosszabbnak látják a hibátlan magyar szöveget, mint a rontottat. A nyelvmodell szépen jelzi, hogy a hibás szöveg nem magyar, míg a helyes szöveg az.

In [15]:
string3_correct = "értékesítő iroda nyitvatartása: hétfő-péntek 8-19h"
string3_ocr = "értíkesítto Iroda m'itfatartdsa: Mjü-ln)Hl<rk 8-19b"

print(bi_ngram_model.metrics_on_string(string3_correct, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string3_ocr, combine_function=multiply))

NameError: name 'bi_ngram_model' is not defined

In [16]:
string4 = "az Átadva az  to^a KeaeáéNt AaaéWkéFedAa cat (atwNtaaA. aww^y"
string5 = "A a L ét • a  ffc Mkm bML A hWiWWHK1**(re a nMtjjww áHő"

print(bi_ngram_model.metrics_on_string(string4, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string5, combine_function=multiply))

NameError: name 'bi_ngram_model' is not defined

Nem magyar nyelvű szöveg felismerésére is jól használhatók a modellek. Míg a magyar szöveg karakterenkénti perplexitása összekapcsoló függvénytől függően 1-4 körül mozog, az angolé 10 felett van. Látványos, hogy a nem emberi nyelven írt OCR-hibás szemét drasztikusan rosszabb perplexitást kap, mint az emberi nyelven írt szöveg.

In [60]:
string_en = "Day Three of PAX East 2022 is done and dusted for another year, which means it's time for another daily round-up video. We had another jam-packed schedule today, checking out Dome Keeper (formerly known as Ludum Dare darling Dome Romantik) over on the Raw Fury booth, before heading over to the PAX Rising Showcase to check out a specially curated selection of upcoming indie games."
print(string_en)
print(bi_ngram_model.metrics_on_string(string_en))
print(bi_ngram_model.metrics_on_string(string_en, combine_function=multiply))

Day Three of PAX East 2022 is done and dusted for another year, which means it's time for another daily round-up video. We had another jam-packed schedule today, checking out Dome Keeper (formerly known as Ludum Dare darling Dome Romantik) over on the Raw Fury booth, before heading over to the PAX Rising Showcase to check out a specially curated selection of upcoming indie games.
(0.37967914438502676, 10.597837912956956)
(0.44919786096256686, 16.246758630922756)


Végül a szemétszűrő. Soronként előbb megnézzük, hogy melyik milyen perplexitást kap.

In [17]:
long_ocr_string = '''HIRDETÉS
Fotó: MTI
UTOLSÓ HELYEINK
FANTASZTIKUS KÓ/no tours
Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
í •f&lMíh•ErKft	iíföfaíj<4^410
TUNÉZIA -október
X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
DJERBA - október
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
korutuzas ♦ üdülés Djerba szigetén,
hotel***:	114 900 Ft/főt ől
HURGHADA -október
X nap hottf**** fp.-val	69 900 H/fótól
botéi***** fp.-val	96 900 Fl/főtől
Nílusi hajóul közvetlen chalerjarultal. 5* m luxuslupóval: 129 900 Ftíőtől
TH-: 2M»-O<.*2. 2M-OH33 TH.: .M2-O.MO. .152-WM* MTI. l«Mr«l 44M. nd.
ajánlaton Portugália .bombaáron* 'épülő, 7 nap teljes p.. X. 14-20.. 21-27 189 900 Ft/« DáFSpanyolorazág-Marokkó. rep. 13 nap
3/4* száll, fél. ill, teljes p. X. 28-XI 9. 349 900 Ft/« Prága-Kariovy Vary, busz 5 nap fp„ X. 19-23.39 900-47 900 Ft/« München, Sörfesztivál, busz, 3 nap reggeli, IX. 27-29.. X. 4-6 29 900 R/«
Cseh várak, városok ét kastélyok, busz, 7 nap reggelt. X. 22-28.. 54 900 Ft/R Rótna-Nápoly-Pompejl, busz, 6 nap reggeli, X, 22-27 66 900 R/W Advent Steyrben, busz, 2 nap fp„ XI. 30.. XII. 1 . XII. 14 . 25 900 R/K Szilveszter az OPATIJAI RIVIERÄN busz, 4 nap fp„ XII. 30 53 900 Ft/« ♦ kdt. szilv. vacsora ' jelentkezés' szekszárdi irodánknál (74)* 5ÍV-955; és több száz vidéki és budapesti partnerünknél toltour@toln8.net, www.toltour.lolna.net
Eng. szám R01963 1999
Prwim Póton Laak
– Fittségi kompiét tesztelés (aUckepevseg. «leti mozjékoaysag)
– Csoportot es egyéni tiiucsadzs (alzklonnilzsrol, testi panaszok- ml eWmodroi, helyes táplálkozásról)
TAéton pArari, hi^wlóáá. ftáadun bntfto. ma. euaa. ftanz^UKKJ.
HASTAMCTANULÁ8 |«t «rakat Joli Jtott	tetor tartja)
Hbrlmcv krtagm ahaayoMi trabaUaa
PEJKÓ Lovaspanzió. Csongrád Mobd: 106-201921-8004«Tel !ai. (06-1) 349-3650
* A szeptemberi időponthoz további 10% engedmény'
A Kereskedelmi és Hitelbank Rt letétkezelésében lévő alapok nettó eszközérték Ft (egy jegy re jutó nettó eszközérték) adatai:
2002. október 4-én:
Aranyhal 359 906 570 (2,584)
2002. október 2-án:
Első Hazai Lakásalap 51 789 494 (9 691)
Mély fájdalommal tudatjuk mindazokkal, akik ismerték és szerették, hogy KÜSTEL RICHÁRD oki. elektromérnök, közgazdász életének 78. évében, hősiesen viselt súlyos betegségben 2002. szeptember 19-én elhunyt.
Végső búcsúztatása a Farkasréti temetőben lesz október 10-én 10.30 órakor. A gyászoló család.
A szeptember 28-al keresztrejtvény nyertese:
Zics Lajos, 1135 Budapest
Nyereményét, a Nagy Képes Világőrténet, és a Magyarország családjai CD-Romot, postázzuk!
Gratulálunk a nyereményhez!'''

In [62]:
for line in long_ocr_string.split('\n'):
    if len(line) < 9:
        continue
    print(line)
    print('\t', bi_ngram_model.metrics_on_string(line, combine_function=multiply))

Fotó: MTI
	 (0.0, 2.6380338031396504)
UTOLSÓ HELYEINK
	 (0.42857142857142855, 33.301556438236425)
FANTASZTIKUS KÓ/no tours
	 (0.3125, 103.46043610157908)
Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
	 (0.6271186440677966, 6.3401560317263606)
í •f&lMíh•ErKft	iíföfaíj<4^410
	 (0.045454545454545456, 47118715.74202395)
TUNÉZIA -október
	 (0.5, 12.210995846406016)
X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
	 (0.2926829268292683, 1401.7067711706945)
DJERBA - október
	 (0.625, 14.955936618620552)
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
	 (0.3939393939393939, 1304.1916205091559)
korutuzas ♦ üdülés Djerba szigetén,
	 (0.4444444444444444, 64.1692913931725)
hotel***:	114 900 Ft/főt ől
	 (0.3684210526315789, 1174.0245871020159)
HURGHADA -október
	 (0.3333333333333333, 31.23847172618334)
X nap hottf**** fp.-val	69 900 H/fótól
	 (0.2, 2000.3507414

Ezek alapján legyen mondjuk 5 a küszöbérték.

In [63]:
for line in long_ocr_string.split('\n'):
    if len(line) < 9:
        continue
    perpl = bi_ngram_model.metrics_on_string(line, combine_function=multiply)[1]
    if perpl < 5:
        print(line)
        print('\t', perpl)

Fotó: MTI
	 2.6380338031396504
* A szeptemberi időponthoz további 10% engedmény'
	 1.5648542356726869
A Kereskedelmi és Hitelbank Rt letétkezelésében lévő alapok nettó eszközérték Ft (egy jegy re jutó nettó eszközérték) adatai:
	 3.0391442471613606
2002. október 4-én:
	 1.439625993835486
2002. október 2-án:
	 1.1880397121979542
Mély fájdalommal tudatjuk mindazokkal, akik ismerték és szerették, hogy KÜSTEL RICHÁRD oki. elektromérnök, közgazdász életének 78. évében, hősiesen viselt súlyos betegségben 2002. szeptember 19-én elhunyt.
	 2.479004951096471
Végső búcsúztatása a Farkasréti temetőben lesz október 10-én 10.30 órakor. A gyászoló család.
	 1.6209947169020056
A szeptember 28-al keresztrejtvény nyertese:
	 2.0855193054917556
Zics Lajos, 1135 Budapest
	 2.4226017658186008
Nyereményét, a Nagy Képes Világőrténet, és a Magyarország családjai CD-Romot, postázzuk!
	 3.661845271876318
Gratulálunk a nyereményhez!
	 1.3019049228616995


In [64]:
for line in long_ocr_string.split('\n'):
    if len(line) < 9:
        continue
    perpl = bi_ngram_model.metrics_on_string(line, combine_function=multiply)[1]
    if perpl > 5:
        print(line)
        print('\t', perpl)

UTOLSÓ HELYEINK
	 33.301556438236425
FANTASZTIKUS KÓ/no tours
	 103.46043610157908
Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
	 6.3401560317263606
í •f&lMíh•ErKft	iíföfaíj<4^410
	 47118715.74202395
TUNÉZIA -október
	 12.210995846406016
X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
	 1401.7067711706945
DJERBA - október
	 14.955936618620552
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
	 1304.1916205091559
korutuzas ♦ üdülés Djerba szigetén,
	 64.1692913931725
hotel***:	114 900 Ft/főt ől
	 1174.0245871020159
HURGHADA -október
	 31.23847172618334
X nap hottf**** fp.-val	69 900 H/fótól
	 2000.3507414559447
botéi***** fp.-val	96 900 Fl/főtől
	 2039.3918782027094
Nílusi hajóul közvetlen chalerjarultal. 5* m luxuslupóval: 129 900 Ftíőtől
	 55.57413745768618
TH-: 2M»-O<.*2. 2M-OH33 TH.: .M2-O.MO. .152-WM* MTI. l«Mr«l 44M. nd.
	 119269.42627733572
a

Ez így szerintem kielégítő eredmény. Úgy tűnik, valami nem jó a padding opcióban, nagyon elrontja a padding nélkül hibátlannak ítélt sorok mért perplexitását is. Bug lehet benne valahol, majd keressem meg alkalomadtán.

## Perplexitás mérése viszonylag nagy fájlon

In [65]:
with open("test_files/1.press_hu_nem_007.txt", encoding='utf-8') as infile:
    very_long_string = infile.read()

In [66]:
print(len(very_long_string))
print(len(very_long_string.split(' ')))

9011378
1138944


Tehát durván egymillió szó.

In [67]:
%%time
bi_ngram_model.metrics_on_string(very_long_string)

CPU times: user 2h 2min 59s, sys: 540 ms, total: 2h 2min 59s
Wall time: 2h 3min 4s


(0.7690967078257801, 2.820009236706759)

In [68]:
%%time
bi_ngram_model.metrics_on_string(very_long_string, combine_function=multiply)

CPU times: user 2h 2min 44s, sys: 448 ms, total: 2h 2min 45s
Wall time: 2h 2min 49s


(0.832808773804649, 1.706981702239145)

Tehát bő 1 millió szóra 2 óra alatt fut le, ez praktikusan nem igazán használható, ellenben a pontossága nagyon jó.
Arra is érdemes gondolni, hogy a perplexitás elméleti minimuma 1 (az egy olyan nyelvmodell és szöveg, ahol a modell mindig teljes biztonsággal és helyesen meg tudja mondani, hogy mi a következő elem), amihez az 1,7 már egész szépen kezd közelíteni.

# Kétirányú LSTM

A modell példányosításához szükség van egy input- és egy outputkarakter-kódoló objektumra, ezekből a konstruktor előállít egy `lstm_model.BiLSTM_Encoder` objektumot, és azt adattagként tárolja a modellobjektumban. Emellett meg kell adni, hogy a bal és a jobb kontextust feldolgozó két LSTM-réteg hány karaktert dolgozzon fel, tehát hány lépést tegyen meg, mielőtt kiírná outputként az állapotát.

Adott esetben a 15, 15 azt jelenti, hogy a megjósolandó középső karaktertől balra 15 karaktert olvas be (előre), és jobbra is 15 karaktert olvas be (visszafele).

In [2]:
import tensorflow as tf
import lstm_model
from encode_characters import InputEncoder, OutputEncoder, character_to_features

# Ha a következő két sor ki van kommentálva, és a Tensorflow-telepítés támogatja a GPU-n
# futtatást, akkor a modell GPU-n fog futni, ami sokkal gyorsabb.
# Ha nincs kikommentálva, akkor CPU-n fut, mint ahogy akkor is, ha nincs GPU, vagy
# ha a Tensorflow úgy lett telepítve, hogy nem használja a meglévő GPU-t.
#tf.config.set_visible_devices([], 'GPU')
#visible_devices = tf.config.get_visible_devices()

input_enc = InputEncoder(file="input_encoder.json")
output_enc = OutputEncoder(file="output_encoder.json")

bilstm_model = lstm_model.BiLSTM_Model(input_encoder=input_enc, output_encoder=output_enc,
                                       left_context=15, right_context=15)

2022-11-18 01:38:47.418600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 01:38:48.516789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 01:38:48.536832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 01:38:48.537307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

Model: "BiLSTM_Model_v0.2.0>w_15-15_lstm_512_dense_512_dropout_0>>2022-11-18_01.38.48"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 mask_left (Masking)            (None, 15, 80)       0           ['input_left[0][0]']             
                                                                                                  
 mask_right (Masking)           (None, 15, 80)       0           ['input_right[0][0]']            
                      

Ha nem új üres modellt akarunk betanítani, akkor be lehet tölteni egy korábban mentett modellt a `BiLSTM_Model.load()` metódussal, aminek a modell elérési útvonalát, valamint az input- és outputkarakter-kódolót kell átadni argumentumként. Az utóbbira azért van szükség, mert a mentés csak magát a Keras-modellt menti el, a `BiLSTM_Model` objektum egyéb adattagjait - egyebek mellett a két karakterkódolót - nem. Értelemszerűen a mentett modell betöltésekor ugyanolyan kódolóobjektumokat kell átadni a `load()` metódusnak, mint amilyenekkel be lett tanítva a modell eredetileg (tehát ugyanazt a nyers karaktert ugyanarra a numerikus vektorra fordítsa le).

Adott esetben az itt betöltött modell az 512 dimenziós (tehát ekkorák egyenként az LSTM-rétegek és az a teljesen kapcsolt réteg, ami ezeket követi a feldolgozásban. Az viszonylag kézenfekvő, hogy a két LSTM-réteg egyforma nagy legyen, de az semmiből se következik, hogy a dense is ekkora legyen, csak az egyszerűség kedvéért választottam ezt.

A végén az összefoglaló kiírja, hogy hány iteráción át lett tanítva a modell. Egy iteráció egy batch feldolgozása, egy batch mérete pedig az alapértelmezett beállítás volt, ami 256. Tehát az 537 ezer azt jelenti, hogy kb. 140 millió karakter, tehát durván 20 millió szó lett feldolgozva a tanítókorpuszból - ami a korpusz egytizede, tehát egytized epoch a teljes tanító adathalmazon - amikor leállítottam a tanítást. (Tanult volna még tovább, csak ennyi a demózáshoz egyelőre elég volt.)

In [3]:
bilstm_model = lstm_model.BiLSTM_Model.load('bilstm_model_512.h5',
                                            input_enc, output_enc)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_512>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 512)          1214464     ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 512)          1214464     ['input_right[0][0]']            
                                                

## A sztringkódoló

A tanításra és predikcióra szolgáló inputszövegeket a kódolóobjektum alakítja át numerikus Numpy-tömbökké, amik átadhatóak bemenetként a Keras-modellnek. A fenti összefoglalóban látszik, hogy a modell két darab k x 15 x 80 dimenziós tenzort vár bemenetként, ahol k az adatpontok (tehát a feldolgozandó célkarakterek) száma, 15 a kontextusablak szélessége, 80 pedig az inputkarakterek kódjának a hossza.

A teljes inputkarakter-kódtáblázat így épül fel olvasható formában:

In [4]:
import pandas as pd

FEATURE_NAMES = ['normalised', 'letter', 'upper', 'long', 'other_diacritic',
                'space', 'dash', 'digit']

ch_features = []
for ch in bilstm_model.encoder.input_encoder.ch_encodings.keys():
    features = list(character_to_features(ch))
    normalised_ch = features[0]
    if normalised_ch != '\u0000' and bilstm_model.encoder.input_encoder.num_code_dict.get(normalised_ch, 0) == 0:
        features[0] = '\u25a1'
    ch_features.append(features)

dframe = pd.DataFrame(ch_features,
                      index=list(bilstm_model.encoder.input_encoder.ch_encodings.keys()),
                      columns=FEATURE_NAMES)
print(dframe.to_string())

   normalised  letter  upper   long  other_diacritic  space   dash  digit
                False  False  False            False  False  False  False
                False  False  False            False   True  False  False
e           e    True  False  False            False  False  False  False
a           a    True  False  False            False  False  False  False
t           t    True  False  False            False  False  False  False
l           l    True  False  False            False  False  False  False
s           s    True  False  False            False  False  False  False
n           n    True  False  False            False  False  False  False
k           k    True  False  False            False  False  False  False
r           r    True  False  False            False  False  False  False
i           i    True  False  False            False  False  False  False
z           z    True  False  False            False  False  False  False
o           o    True  False  False   

Az első két láthatatlan karakter a padding (`\u0000`) és a sima szóköz karakter.

A sorok címkéi maguk az inputkarakterek, ahogy az inputszövegben szerepelnek.

A "normalizált karakter" oszlopban szereplő fehér négyzet az OOV-karaktert jelöli, a nagyon ritka jeleket így kódolja az inputkódoló (míg a nagyon ritka betűket simán lenormalizálja a megfelelő mellékjel nélküli betűre).

Miután ez megvan, az inputkódoló még a normalizált karaktereket átkódolja egy-egy one-hot kóddá, így például az *á* karakter kódja így néz ki:

In [5]:
print(bilstm_model.encoder.input_encoder.num_code_dict['a'])
print(bilstm_model.encoder.input_encoder.encode('á'))
print(len(bilstm_model.encoder.input_encoder.encode('á')))

3
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 0.]
80


Az elején az egyes a normalizált *a* numerikus kódjának (3) a one-hot kódolása, a végén a két egyes pedig a 'betű' és a 'hosszú' bináris jegy.

Így jönnek ki tehát a 80 dimenziós inputkarakter-vektorok.

Szintén a fenti összefoglalóban látszik, hogy az outputvektor 113 dimenziós. Az sima one-hot vektor.

In [6]:
print(bilstm_model.encoder.output_encoder.encode('á'))
print(len(bilstm_model.encoder.output_encoder.encode('á')))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
113


A sztringkódoló lekódolja az inputsztring összes karakterét az input- és outputkódoló objektum segítségével egy inputkódolt és egy outputkódolt mátrixszá, tehát lesz egy k x 80 és egy k x 113 elemű mátrix, ahol k az inputsztring hossza. Aztán ezen a mátrixon végigléptet egy-egy soronként haladva egy megfelelő szélességű (bal 15 + középen 1 + jobb 15 = 31) ablakot, és az inputmátrix megfelelő két (jobb és bal) 15 x 80 elemű szeletét és az outputmátrix 1 x 113 elemű egyetlen sorát beleteszi a két darab k x 15 x 80-as tenzor és a k x 113-as mátrix i-edik sorába, ahol i az éppen feldolgozott célkarakter (outputkarakter) indexe az inputsztringben.

Ezt a `BiLSTM_Encoder.encode()` metódus végzi el:

In [7]:
test_string = 'Ez egy próbasztring.'
xleft, xright, y = bilstm_model.encoder.encode(test_string, padded=True)
print(len(test_string))
print(xleft.shape)
print(xright.shape)
print(y.shape)

20
(20, 15, 80)
(20, 15, 80)
(20, 113)


Amellett, hogy simán visszaad értékként három ilyen Numpy-tömböt, egy másik lehetőség, hogy a híváskor adunk át neki a `call_by_reference` argumentum értékeként három ilyen tömböt, és a kódoló ezekbe egyszerűen beleír, és nem ad vissza semmit. Ez lényegében akkor hasznos, ha mindig egy batch-nyi (pl. 128) hosszúságú sztringet akarunk lekódoltatni a kódolóval egyszerre, ilyenkor ugyanis nem kell minden híváskor újra és újra lefoglalnia a Numpy-tömbnek a tárterületet és megkonstruálni az objektumot hozzá, hanem simán beleír a már lefoglalt területre.

Így mérhetően, 10-20 százalékkal gyorsabban fut a tanítás, ezért a modell betanítására szolgáló `BiLSTM_Model.train()` metódus által használt `BiLSTM_Sequence` objektum, ami gyakorlatilag batch-nyi darabokra bontja az inputszöveget, és ezeket a szeleteket adja át egyenként a sztringkódolónak, ezt a módszert használja a metódussal való adatcserére.

In [8]:
import numpy as np

xleft = np.zeros([20,15,80])
xright = np.zeros([20,15,80])
y = np.zeros([20,113])

bilstm_model.encoder.encode(test_string, call_by_reference=(xleft, xright, y), padded=True)
bilstm_model.encoder.decode(y)

'Ez egy próbasztring.'

In [9]:
test_string2 = "Ez pedig egy másik. "
print(len(test_string2))

bilstm_model.encoder.encode(test_string2, call_by_reference=(xleft, xright, y), padded=True)
bilstm_model.encoder.decode(y)

20


'Ez pedig egy másik. '

## Kényelmi metódusok

Miután mind a tanításra, mind a predikcióra megvannak a szükséges kényelmi metódusok, általában nem kell bajlódnunk a sztringkódoló használatával.

Ezek a metódusok lehetőség szerint megegyeznek mind névre, mind szignatúrára az n-gram-modellek megfelelő funkciójú metódusaival.

Az `estimate_alternatives()` dictionaryt ad vissza, aminek a kulcsai a lehetséges outputkaraktererek, az értékei a hozzájuk tartozó becsült feltételes valószínűségek. Ez alapértelmezés szerint a neki átadott sztringnek a modellre beállított bal kontextushossznak megfelelő részét (adott esetben 15 karaktert) kezeli bal kontextusként, a következő karaktert célként, a következő jobb kontextushossznyi (adott esetben 15) karaktert jobb kontextusként, és a célkarakterre adja vissza a predikciókat.

In [10]:
%%time

def select_window(string, t_index):
    start_index = t_index - bilstm_model.encoder.left_context
    end_index = t_index + bilstm_model.encoder.right_context + 1
    print(string[start_index:t_index])
    print(bilstm_model.encoder.left_context * ' ' + long_string_1[t_index])
    print((bilstm_model.encoder.left_context + 1) * ' ' + long_string_1[t_index + 1:end_index])
    return string[start_index:end_index]

print(long_string_1)
print()
target_index = 25

print(select_window(long_string_1, target_index)) # csak szemléltetésként, hogy mit csinál az estimate_alternatives

bilstm_model.estimate_alternatives(long_string_1, target_index)

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.

atlan a parkolá
               s
                i helyzet a lak
atlan a parkolási helyzet a lak
CPU times: user 480 ms, sys: 253 ms, total:

2022-11-18 01:39:16.098007: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2022-11-18 01:39:16.178990: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


{'□': 1e-13,
 ' ': 1e-13,
 'e': 1e-13,
 'a': 1e-13,
 't': 1.9480823e-11,
 'l': 1e-13,
 's': 1.0,
 'n': 1e-13,
 'k': 3.8933787e-12,
 'r': 1.0234147e-13,
 'i': 1e-13,
 'z': 1e-13,
 'o': 1e-13,
 'á': 1e-13,
 'é': 1e-13,
 'g': 1e-13,
 'm': 4.343061e-13,
 'y': 1e-13,
 'b': 1e-13,
 'd': 1e-13,
 'v': 1e-13,
 ',': 1e-13,
 'h': 1e-13,
 'j': 1e-13,
 'p': 1e-13,
 'ó': 1e-13,
 'ö': 1e-13,
 'u': 1e-13,
 'ő': 1e-13,
 '.': 1e-13,
 '\n': 1e-13,
 'f': 1e-13,
 'c': 1e-13,
 'í': 1e-13,
 'ü': 1e-13,
 'A': 1e-13,
 '-': 1e-13,
 'ú': 1e-13,
 '0': 1e-13,
 'M': 1e-13,
 '1': 1e-13,
 'S': 1e-13,
 'ű': 1e-13,
 '2': 1e-13,
 'E': 1e-13,
 'B': 1e-13,
 'K': 1e-13,
 '"': 1e-13,
 'T': 1e-13,
 'P': 1e-13,
 ':': 1e-13,
 'N': 1e-13,
 'F': 1e-13,
 'H': 1e-13,
 '5': 1e-13,
 '3': 1e-13,
 'V': 1e-13,
 '9': 1e-13,
 'I': 1e-13,
 ')': 1e-13,
 '(': 1e-13,
 'C': 1e-13,
 '4': 1e-13,
 'L': 1e-13,
 'D': 1e-13,
 'G': 1e-13,
 'R': 1e-13,
 '6': 1e-13,
 '8': 1e-13,
 '7': 1e-13,
 'O': 1e-13,
 'J': 1e-13,
 'Z': 1e-13,
 'x': 1e-13,
 'w': 1e

In [11]:
target_index = 169

print(select_window(long_string_1, target_index))

bilstm_model.estimate_alternatives(long_string_1, target_index)

ra egyre többsz
               ö
                r okoz elemi pr
ra egyre többször okoz elemi pr


{'□': 1e-13,
 ' ': 1e-13,
 'e': 1.3131561e-06,
 'a': 3.6015937e-11,
 't': 1e-13,
 'l': 1e-13,
 's': 1e-13,
 'n': 1e-13,
 'k': 1e-13,
 'r': 1e-13,
 'i': 5.3338215e-13,
 'z': 1e-13,
 'o': 2.666902e-07,
 'á': 4.525107e-09,
 'é': 2.6785518e-10,
 'g': 1e-13,
 'm': 1e-13,
 'y': 1e-13,
 'b': 1e-13,
 'd': 1e-13,
 'v': 1e-13,
 ',': 1e-13,
 'h': 1e-13,
 'j': 1e-13,
 'p': 1e-13,
 'ó': 3.5915194e-13,
 'ö': 0.99999845,
 'u': 1.917708e-12,
 'ő': 1e-13,
 '.': 1e-13,
 '\n': 1e-13,
 'f': 1e-13,
 'c': 1e-13,
 'í': 1e-13,
 'ü': 1e-13,
 'A': 1e-13,
 '-': 1e-13,
 'ú': 1e-13,
 '0': 1e-13,
 'M': 1e-13,
 '1': 1e-13,
 'S': 1e-13,
 'ű': 1e-13,
 '2': 1e-13,
 'E': 1e-13,
 'B': 1e-13,
 'K': 1e-13,
 '"': 1e-13,
 'T': 1e-13,
 'P': 1e-13,
 ':': 1e-13,
 'N': 1e-13,
 'F': 1e-13,
 'H': 1e-13,
 '5': 1e-13,
 '3': 1e-13,
 'V': 1e-13,
 '9': 1e-13,
 'I': 1e-13,
 ')': 1e-13,
 '(': 1e-13,
 'C': 1e-13,
 '4': 1e-13,
 'L': 1e-13,
 'D': 1e-13,
 'G': 1e-13,
 'R': 1e-13,
 '6': 1e-13,
 '8': 1e-13,
 '7': 1e-13,
 'O': 1e-13,
 'J': 1e-1

Érdemes megemlíteni, hogy mivel a softmax függvény sok nagyon kis valószínűséget 0-ra kerekít, és azokkal nem lehet perplexitást (vagy bármilyen logaritmust) számolni, betettem az LSTM-modellbe egy alsó küszöböt a valószínűségekre, ami láthatóan 1e-13. Ebben a nagyságrendben úgyis már minden lebegőpontos számítás nagyon pontatlan, és ha ezekből 100-at összeadunk (ti. annyiféle karakter van), akkor az még mindig csak 1e-11, tehát ez a lépés a valószínűségi eloszlást a legkevésbé sem rontja el. Ugyanezt talán célszerű lenne meglépni az n-gram-modellnél is, hogy következetes legyen.

Ha nem adunk meg `target_index` paramétert, az `estimate_alternatives()` abból indul ki, hogy ablaknyi hosszúságú inputot kapott, és a középső (adott esetben 16.) karakterre számítja a predikciókat.

In [12]:
print(select_window(long_string_1, 15))

bilstm_model.estimate_alternatives(long_string_1)

A „Tarthatatlan
                
                a parkolási hel
A „Tarthatatlan a parkolási hel


{'□': 1e-13,
 ' ': 0.9994504,
 'e': 1.8564586e-08,
 'a': 3.781017e-09,
 't': 7.418678e-08,
 'l': 2.7217442e-09,
 's': 5.664817e-09,
 'n': 4.9825055e-09,
 'k': 1.7309057e-07,
 'r': 6.3307685e-05,
 'i': 4.4132644e-08,
 'z': 1.4452097e-09,
 'o': 3.0805187e-09,
 'á': 1.574226e-11,
 'é': 3.653883e-09,
 'g': 1.10469106e-10,
 'm': 3.7523376e-09,
 'y': 2.1793449e-10,
 'b': 8.6091095e-06,
 'd': 4.3269752e-10,
 'v': 3.11291e-09,
 ',': 1e-13,
 'h': 1e-13,
 'j': 1.4514558e-05,
 'p': 7.3954762e-09,
 'ó': 3.8564824e-12,
 'ö': 1e-13,
 'u': 2.7817948e-10,
 'ő': 1e-13,
 '.': 1e-13,
 '\n': 3.0707977e-13,
 'f': 7.7155754e-11,
 'c': 2.3922822e-10,
 'í': 3.2126274e-11,
 'ü': 1e-13,
 'A': 1e-13,
 '-': 0.0004628848,
 'ú': 1e-13,
 '0': 1e-13,
 'M': 1e-13,
 '1': 1e-13,
 'S': 1e-13,
 'ű': 1e-13,
 '2': 1e-13,
 'E': 1e-13,
 'B': 1e-13,
 'K': 1e-13,
 '"': 1e-13,
 'T': 1e-13,
 'P': 1e-13,
 ':': 1e-13,
 'N': 1e-13,
 'F': 1e-13,
 'H': 1e-13,
 '5': 1e-13,
 '3': 1e-13,
 'V': 1e-13,
 '9': 1e-13,
 'I': 1e-13,
 ')': 1e-13

A `predict_next()` függvény egy megadott bal és jobb sztring közé eső karaktert jósol. Ha a bal vagy jobb sztring rövidebb a kontextushossznál, akkor paddingkaraktereket ad hozzá az elejére, illetve végére, ha pedig hosszabb bármelyik, akkor a balnak csak az utolsó 15, illetve a jobbnak csak az első 15 karakterét nézi.

In [13]:
target_index = 169

print(select_window(long_string_1, target_index))

bilstm_model.predict_next('ra egyre többsz', 'r okoz elemi pr', m=5)

ra egyre többsz
               ö
                r okoz elemi pr
ra egyre többször okoz elemi pr


['ö', 'e', 'o', 'á', 'é']

Az `estimate_prob()` a célindexen levő (vagy alapértelmezés szerint a teljes kontextusnyi inputsztringben a középső) karakter becsült valószínűségét adja meg, a `pred_estimate()` pedig azt, hogy a modell szerint mi a legvalószínűbb karakter abban a pozícióban, majd a tényleges karakter valószínűségét.

In [14]:
print(bilstm_model.estimate_prob('ra egyre többször okoz elemi pr'))
print(bilstm_model.pred_estimate('ra egyre többször okoz elemi pr'))

0.99999845
('ö', 0.99999845)


In [15]:
window = select_window(long_string_1, 89)
print(window)
print(bilstm_model.estimate_prob(long_string_1, 89))
print(bilstm_model.pred_estimate(long_string_1, 89))
print(bilstm_model.predict_next(window[:15], window[16:], m=5))

etesen bemutatt
               u
                k, hogy a Budap
etesen bemutattuk, hogy a Budap
0.104415834
('á', 0.104415834)
['á', 'u', 'a', 'é', 'o']


A `metrics_on_string()` kiszámítja a teljes inputsztringre (választhatóan automatikus paddinggel) a karakterenkénti pontosságot és perplexitást.

Itt válik el a GPU és a CPU teljesítménye egymástól. Ez alábbi GPU-n nálam 1 másodperc alatt fut le, ami valamivel gyorsabb, mint a kétirányú n-gram-modell, CPU-n viszont 35 másodperc alatt, ami használhatatlanul lassú. A 64 dimenziós LSTM- és dense-rétegeket tartalmazó modell (amiben csak 90 ezer paraméter van) még elviselhető sebességgel prediktál rövid inputokra, de az ilyen nagy modell (3 millió paraméter) már nem.

In [16]:
%%time
print(bilstm_model.metrics_on_string(long_string_1))
print(bilstm_model.metrics_on_string(long_string_1, padded=True))
print(bilstm_model.metrics_on_string(long_string_2))
print(bilstm_model.metrics_on_string(long_string_2, padded=True))

(0.9734299516908212, 1.0837712076183974)
(0.9708624708624709, 1.1057401867751655)
(0.9572327044025157, 1.1823819262171171)
(0.9563636363636364, 1.1842971470568495)
CPU times: user 332 ms, sys: 40.8 ms, total: 373 ms
Wall time: 346 ms


Összehasonlításképpen a kétirányú n-gram-modell teljesítménye ugyanezeken:

In [80]:
%%time
print(bi_ngram_model.metrics_on_string(long_string_1))
print(bi_ngram_model.metrics_on_string(long_string_1, padded=True))
print(bi_ngram_model.metrics_on_string(long_string_2))
print(bi_ngram_model.metrics_on_string(long_string_2, padded=True))

(0.8058823529411765, 2.630060632382658)
(0.8018648018648019, 2.7088778690523965)
(0.7356181150550796, 2.982173827104181)
(0.7345454545454545, 3.011807170746155)
CPU times: user 2.72 s, sys: 0 ns, total: 2.72 s
Wall time: 2.72 s


Opcionálisan kiíratható a modell által előrejelzett sztring is.

In [17]:
%%time
print(long_string_1)
print()
print(bilstm_model.metrics_on_string(long_string_1, print_string=True))
print()
print(bilstm_model.metrics_on_string(long_string_1, print_string=True, padded=True))

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.

 a parkolási helyzet a lakótelepekwn. című cikkünkben részletesen bemutatják, hogy a Budapest sűrűn lakott városrészekben élő autósok számár

In [18]:
%%time
print(long_string_2)
print()
print(bilstm_model.metrics_on_string(long_string_2, print_string=True))
print()
print(bilstm_model.metrics_on_string(long_string_2, print_string=True, padded=True))

Tavaly októberben adtunk hírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Joshi Bharattal alkot egy párt. Most viszont a Story alig pár hónappal később arról számolt be, hogy véget ért kettejük kapcsolata, a lap információi szerint Joshi Bharat már el is költözött abból a budai villából, ahol eddig együtt éltek.
Kelemen Beatrix hosszú évekig élt házasságban Mészáros Lőrinccel. A pár 2020 őszén adta be a váló pert, a részletekről, így a vagyonról is hamar megegyeztek. Egy évvel később már arról lehetett hallani, hogy Kelemen Beatrix Joshi Bharattal alkot egy párt.
Közben pedig, ki ne tudná, Mészáros Lőrinc is újra szerelmes lett és már újra is nősült. Az ország egyik leggazdagabb embere a TV2 korábbi műsorvezetőjével, Várkonyi Andreával kötötte össze az életét.

en adjunk bírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Josei Searattal alkot egy párt. Most viszont a Story alig p

A `string_perplexity()` csak a perplexitást számolja, padding nélkül.

In [19]:
print(bilstm_model.string_perplexity(long_string_1))
print(bilstm_model.string_perplexity(long_string_2))

1.0837712076183974
1.1823819262171171


A `predict_on_string()` a megadott inputsztring megadott pozíciójára (alapértelmezés szerint a bal kontextus után következő elsőre) adja vissza a Keras-modell `predict()` metódusa által számolt predikcióvektort.

Ez gyakorlatilag ugyanaz, mintha az inputsztringre futtatnánk a `predict()` metódust, csak nem a felhasználónak kell bajlódnia a bal és a jobb X inputmátrix lekódolásával a sztring alapján, és a `predict()` függvény által visszaadott mátrixból kiveszi az egyetlen sort mint vektort, valamint a 0 és az alsó valószínűségi küszöb alá eső egyéb nagyon kis valószínűségeket kicseréli az alsó valószínűségi küszöbre.

In [21]:
pr_string = 'ra egyre többször okoz elemi pr'

left_X, right_X, _ = bilstm_model.encoder.encode(pr_string, padded=False)
keras_pred = bilstm_model.model.predict(x=[left_X, right_X], verbose=0)
print(keras_pred.shape)

convenient_pred = bilstm_model.predict_on_string(pr_string)
print(convenient_pred.shape)
print()

print(convenient_pred)  # ez a predikcióvektor

(1, 113)
(113,)

[9.9999998e-14 9.9999998e-14 1.3131561e-06 3.6015937e-11 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 5.3338215e-13 9.9999998e-14 2.6669019e-07 4.5251070e-09 2.6785518e-10
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 3.5915194e-13 9.9999845e-01 1.9177080e-12 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14
 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9999998e-14 9.9

In [22]:
# Ezekben az elemekben különbözik a metódus által visszaadott predikcióvektor a Keras
# által visszaadottól. Az alacsonyabb valószínűségek ki lettek cserélve 1e-13-ra, más
# nem változott
print(keras_pred[0][keras_pred[0] != convenient_pred])
print(sum(convenient_pred[keras_pred[0] != convenient_pred]))
                    # ennyit módosult a becsült valószínűségek összege a csere által
print()

print(sum(keras_pred[0])) # a predikciók összege eleve nem 1
print(1 - sum(keras_pred[0])) # ennyiben tér el az 1-től a softmax lebegőpontos hibája miatt a predikciók összege

[0.0000000e+00 6.4316732e-23 5.3506234e-32 0.0000000e+00 5.2040062e-28
 1.1251697e-37 1.2896772e-32 1.2655005e-31 1.8081938e-32 4.9986114e-37
 0.0000000e+00 1.9153030e-34 0.0000000e+00 4.4585400e-31 0.0000000e+00
 0.0000000e+00 3.8535153e-38 0.0000000e+00 0.0000000e+00 4.8827451e-14
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 2.0631983e-14
 1.5915439e-23 0.0000000e+00 0.0000000e+00 1.1436215e-14 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.5943519e-24 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.000

Látható, hogy a valószínűségeknek a küszöbértékre való cseréje a lebegőpontos számítás inherens hibájához képest elhanyagolható változást okoz, tehát tökéletesen ártalmatlan.

## Tanítás, kiértékelés

A modell tanítására a `BiLSTM_Model.train()` metódust érdemes használni. Ennek egy olyan iterálható objektumot kell átadni, amelynek elemei szövegsztringek, tehát pl. sztringek (akár egyelemű) listáját. Az összes ilyen sztring balra és jobbra paddinget kap, megtörténik a numerikus kódolása és a batch-ekre bontásuk, és így adódnak át a Keras-modell `fit()` metódusának.

A batch-ek kezelésével egyáltalán nem kell foglalkozni, tehát nem gond, ha az iterálható objektum elemei rövidebbek vagy hosszabbak a batch méreténél, mert a háttérben létrehozott `BiLSTM_Sequence` objektum a rövidebb sztringeket összerakja egy batch-be, a hosszabbakat szétbontja több batch-re, stb.

Opcionálisan megadható egy validálásra szolgáló szöveghalmaz is (ugyanúgy egy iterálható objektum, aminek az elemei szövegsztringek), ezekre számítja minden epoch végén a pontosságot és a perplexitást a Keras. Ha nem adunk meg ilyet, akkor nem validál.

Ha újra futtatjuk a `train()` metódust, a tanítás ott folytatódik, ahol előzőleg abbamaradt. A batch-méretet, validáló halmazt nyugodtan lehet módosítani két futtatás között, illetve értelemszerűen a tanító korpuszt is.

A batch méretével érdemes kísérletezni, hogy milyen mérettel egy epoch mennyi idő alatt fut le, és adott idő alatt mennyit javulnak a metrikák. Nagy batch-méretnél (pl. 4096 és a felett) gyorsan feldolgoz egy-egy epochot, de keveset tanul belőle. Pl. 4096-os batch-méretnél az alábbi tanítókorpuszon egy epoch kb. 2,5 perc alatt lefut, de 5 epoch (összesen 13 perc) alatt jelentősen gyengébb pontosságot és perplexitást ér el, mint 256-os batch-mérettel, egyetlen epoch feldolgozásával nem egészen 12 perc alatt:

In [86]:
bilstm_model_empty = lstm_model.BiLSTM_Model(input_encoder=input_enc, output_encoder=output_enc,
                                             left_context=15, right_context=15)

with open("test_files/2.press_hu_promenad_003_2010.txt", encoding='utf-8') as infile:
    train_corpus = infile.read()

bilstm_model_empty.train(texts=train_corpus.split("\n"),
                         batch_size=4096,
                         validation_texts=[long_string_1, long_string_2],
                         num_epochs=5)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_left (InputLayer)         [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
input_right (InputLayer)        [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
lstm_forward (LSTM)             (None, 64)           37120       input_left[0][0]                 
__________________________________________________________________________________________________
lstm_backward (LSTM)            (None, 64)           37120       input_right[0][0]                
____________________________________________________________________________________________

In [87]:
bilstm_model_empty2 = lstm_model.BiLSTM_Model(input_encoder=input_enc, output_encoder=output_enc,
                                             left_context=15, right_context=15)

bilstm_model_empty2.train(texts=train_corpus.split("\n"),
                         batch_size=256,
                         validation_texts=[long_string_1, long_string_2],
                         num_epochs=1)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_left (InputLayer)         [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
input_right (InputLayer)        [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
lstm_forward (LSTM)             (None, 64)           37120       input_left[0][0]                 
__________________________________________________________________________________________________
lstm_backward (LSTM)            (None, 64)           37120       input_right[0][0]                
____________________________________________________________________________________________

Viszont másfelől ha túl kicsi a batch-méret, akkor adott idő alatt megint kevésbé jól tanul. Például 128-as batch-méret mellett 40 perc tanulás (2 epoch) után kb. ugyanolyan pontosságú és perplexitású modellt kaptam, mint 512-es batch-csel 30 perc (4 epoch) tanulás után, és jelentősen rosszabbat, mint 256-os méret mellett 35 perc (3 epoch) után. Úgy találtam, hogy a 256 vagy 512 batch-mérettel a leghatékonyabb a tanulás, a kettő között nincs határozott különbség.

In [86]:
bilstm_model_empty6 = lstm_model.BiLSTM_Model(input_encoder=input_enc, output_encoder=output_enc,
                                             left_context=15, right_context=15)

bilstm_model_empty6.train(texts=train_corpus.split("\n"),
                         batch_size=256,
                         validation_texts=[long_string_1, long_string_2],
                         num_epochs=3)

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_left (InputLayer)         [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
input_right (InputLayer)        [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
lstm_forward (LSTM)             (None, 64)           37120       input_left[0][0]                 
__________________________________________________________________________________________________
lstm_backward (LSTM)            (None, 64)           37120       input_right[0][0]                
___________________________________________________________________________________________

In [90]:
bilstm_model_empty5 = lstm_model.BiLSTM_Model(input_encoder=input_enc, output_encoder=output_enc,
                                             left_context=15, right_context=15)

bilstm_model_empty5.train(texts=train_corpus.split("\n"),
                         batch_size=512,
                         validation_texts=[long_string_1, long_string_2],
                         num_epochs=4)

Model: "model_47"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_left (InputLayer)         [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
input_right (InputLayer)        [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
lstm_forward (LSTM)             (None, 64)           37120       input_left[0][0]                 
__________________________________________________________________________________________________
lstm_backward (LSTM)            (None, 64)           37120       input_right[0][0]                
___________________________________________________________________________________________

In [88]:
bilstm_model_empty7 = lstm_model.BiLSTM_Model(input_encoder=input_enc, output_encoder=output_enc,
                                             left_context=15, right_context=15)

bilstm_model_empty7.train(texts=train_corpus.split("\n"),
                         batch_size=128,
                         validation_texts=[long_string_1, long_string_2],
                         num_epochs=2)

Model: "model_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_left (InputLayer)         [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
input_right (InputLayer)        [(None, 15, 80)]     0                                            
__________________________________________________________________________________________________
lstm_forward (LSTM)             (None, 64)           37120       input_left[0][0]                 
__________________________________________________________________________________________________
lstm_backward (LSTM)            (None, 64)           37120       input_right[0][0]                
___________________________________________________________________________________________

Ha egy szövegen akarjuk mérni a modell előrejelzésének pontosságát és a perplexitást, akkor normális esetben a `metrics_on_string()` metódust érdemes használni, kivéve, ha nagyon nagy korpuszon akarunk kiértékelni, ugyanis nagyon hosszú szövegek egyszerre nem férnek el kódolva a RAM-ban. Nálam a 64 GB RAM néhány százezer karakter kódolásánál már elfogyott. Ezért ha nagyon nagy szövegen akarunk kiértékelni (tehát nem egy-egy rövid szakasz perplexitására vagyunk kíváncsiak, mint a nyelvfelismerésnél stb., hanem ténylegesen a modell perplexitására egy adott nagyobb korpuszon), akkor ehhez érdemes példányosítani egy `BiLSTM_Sequence` objektumot, és azt átadni a Keras-modell `evaluate()` metódusának. Az felbontja a hosszú sztringet a megadott hosszúságú batch-ekre, és egyébként is jelentősen gyorsabban fut.

Tanulásnál a túl nagy batch-méret nem hasznos, mert kevésbé jól tanul a modell, de kiértékelésnél érdemes minél nagyobb batch-et megadni, mert annál gyorsabb. Arra kell csak vigyázni, hogy ha a modell nagy, akkor nagyon nagy batch-méret nem fér el a GPU memóriájában, ezért ha ezt tapasztaljuk, a batch-méretet kisebbre kell venni. Például a 64 dimenziós kis modellel nálam a 65536 batch-méret is lefutott, de 256 dimenziós felett már 32768 batch-méretre sem volt elég a memória.

In [87]:
with open('test_files/2.press_hu_promenad_003_2011.txt', encoding='utf-8') as infile:
    test_text = infile.read()

test_text_sequence = lstm_model.BiLSTM_Sequence([test_text], batch_size=32768, encoder=bilstm_model_empty.encoder)

bilstm_model_empty.model.evaluate(x=test_text_sequence)

74/74 [==============================] - 62s 836ms/step - loss: 1.7365 - accuracy: 0.5998 - perplexity: 5.1322


[1.7365268468856812, 0.5998282432556152, 5.132189152285671]

In [88]:
test_text_sequence2 = lstm_model.BiLSTM_Sequence([test_text], batch_size=1024, encoder=bilstm_model_empty.encoder)

bilstm_model_empty.model.evaluate(x=test_text_sequence2)

2358/2358 [==============================] - 108s 46ms/step - loss: 1.7365 - accuracy: 0.5998 - perplexity: 5.1322


[1.7365278005599976, 0.5998282432556152, 5.1321891206239165]

## Perplexitás mérése viszonylag nagy fájlon

Az alábbi modellek nem azonos ideig lettek betanítva. A nagyobb modellek kevesebb ideig, a tanítókorpusznak csak egy kis részén, de azon is csak egyszer. A betöltéskor kiírt iterációszámot 256-tal kell szorozni, és ha minden igaz, annyi célkarakteren lett tanítva a modell, ha nem lett a tanítás közben egyszer sem lenullázva a számláló, amiben nem vagyok biztos. Mindenesetre így az összehasonlítás nem teljesen pontos, de aránylag jól érzékelteti, hogy a nagyobb modellek következetesen egyre jobbak (annak dacára is, hogy kevesebbet lettek tanítva). A mérésre használt fájl természetesen nem része a tanítókorpusznak.

Emlékeztetőül: a kétirányú n-gram-modell erre a fájlra bő 2 óra alatt futott le, és a pontosság, illetve perplexitás 0.8328, illetve 1.7070 volt.

In [24]:
bilstm_model.model.name

'BiLSTM_Model_v0.2.0>bilstm_model_512>'

In [27]:
lstm_model.rename_model('bilstm_model_128.h5', 'BiLSTM_Model_v0.2.0>bilstm_model_128>', rename_file=False)
lstm_model.rename_model('bilstm_model_192.h5', 'BiLSTM_Model_v0.2.0>bilstm_model_192>', rename_file=False)
lstm_model.rename_model('bilstm_model_256.h5', 'BiLSTM_Model_v0.2.0>bilstm_model_256>', rename_file=False)
lstm_model.rename_model('bilstm_model_384.h5', 'BiLSTM_Model_v0.2.0>bilstm_model_384>', rename_file=False)

In [26]:
model_64 = lstm_model.BiLSTM_Model.load('bilstm_model_64.h5',
                                            input_enc, output_enc)
with open('test_files/1.press_hu_nem_007.txt', encoding='utf-8') as infile:
    test_text = infile.read()

test_text_sequence = lstm_model.BiLSTM_Sequence([test_text], batch_size=32768, encoder=model_64.encoder)

model_64.model.evaluate(x=test_text_sequence)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_64>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 64)           37120       ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 64)           37120       ['input_right[0][0]']            
                                                 

[0.642272412776947, 0.8194453716278076, 1.8904609498377711]

In [28]:
model_128 = lstm_model.BiLSTM_Model.load('bilstm_model_128.h5',
                                            input_enc, output_enc)
model_128.model.evaluate(x=test_text_sequence)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_128>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 128)          107008      ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 128)          107008      ['input_right[0][0]']            
                                                

[0.36633217334747314, 0.8951641917228699, 1.4351512310440604]

In [29]:
model_192 = lstm_model.BiLSTM_Model.load('bilstm_model_192.h5',
                                            input_enc, output_enc)
model_192.model.evaluate(x=test_text_sequence)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_192>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 192)          209664      ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 192)          209664      ['input_right[0][0]']            
                                                

[0.26114746928215027, 0.9239715337753296, 1.2921716210729335]

In [30]:
model_256 = lstm_model.BiLSTM_Model.load('bilstm_model_256.h5',
                                            input_enc, output_enc)
model_256.model.evaluate(x=test_text_sequence)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_256>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 256)          345088      ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 256)          345088      ['input_right[0][0]']            
                                                

[0.21734358370304108, 0.937168538570404, 1.2367998853006918]

In [31]:
model_384 = lstm_model.BiLSTM_Model.load('bilstm_model_384.h5',
                                            input_enc, output_enc)

test_text_sequence = lstm_model.BiLSTM_Sequence([test_text], batch_size=8192, encoder=model_384.encoder)
model_384.model.evaluate(x=test_text_sequence)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_384>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 384)          714240      ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 384)          714240      ['input_right[0][0]']            
                                                

[0.19447290897369385, 0.9421954154968262, 1.2111127176205565]

In [32]:
model_512 = lstm_model.BiLSTM_Model.load('bilstm_model_512.h5',
                                            input_enc, output_enc)
model_512.model.evaluate(x=test_text_sequence)

Loaded model:
Model: "BiLSTM_Model_v0.2.0>bilstm_model_512>"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 15, 80)]     0           []                               
                                                                                                  
 input_right (InputLayer)       [(None, 15, 80)]     0           []                               
                                                                                                  
 lstm_forward (LSTM)            (None, 512)          1214464     ['input_left[0][0]']             
                                                                                                  
 lstm_backward (LSTM)           (None, 512)          1214464     ['input_right[0][0]']            
                                                

[0.15595199167728424, 0.9543982148170471, 1.1607969183026523]

# LSTM- és n-gram-modellek összehasonlítása

**Normál rövid magyar szövegeken paddinggel vagy nélküle:**

In [95]:
%%time
print(long_string_1)
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=multiply, padded=True))

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.
(0.8623529411764705, 1.5094330610760718)
(0.8578088578088578, 1.5973799384302159)
CPU times: user 1.41 s, sys: 7.69 ms, total: 1.41 s
Wall ti

In [33]:
%%time
print(bilstm_model.metrics_on_string(long_string_1))
print(bilstm_model.metrics_on_string(long_string_1, padded=True))

(0.9734299516908212, 1.0837712076183974)
(0.9708624708624709, 1.1057401867751655)
CPU times: user 162 ms, sys: 26 ms, total: 188 ms
Wall time: 168 ms


Az LSTM-modell nagyjából háromszor olyan gyorsan végzi el a számítást, és sokkal jobb eredményt ad.

In [97]:
%%time
print(long_string_2)
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=multiply, padded=True))

Tavaly októberben adtunk hírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Joshi Bharattal alkot egy párt. Most viszont a Story alig pár hónappal később arról számolt be, hogy véget ért kettejük kapcsolata, a lap információi szerint Joshi Bharat már el is költözött abból a budai villából, ahol eddig együtt éltek.
Kelemen Beatrix hosszú évekig élt házasságban Mészáros Lőrinccel. A pár 2020 őszén adta be a váló pert, a részletekről, így a vagyonról is hamar megegyeztek. Egy évvel később már arról lehetett hallani, hogy Kelemen Beatrix Joshi Bharattal alkot egy párt.
Közben pedig, ki ne tudná, Mészáros Lőrinc is újra szerelmes lett és már újra is nősült. Az ország egyik leggazdagabb embere a TV2 korábbi műsorvezetőjével, Várkonyi Andreával kötötte össze az életét.
(0.7968176254589964, 1.7982801627641019)
(0.7951515151515152, 1.8481860709167175)
CPU times: user 1.41 s, sys: 0 ns, total: 1.41 s
Wall time: 1.41 s


In [34]:
%%time
print(bilstm_model.metrics_on_string(long_string_2))
print(bilstm_model.metrics_on_string(long_string_2, padded=True))

(0.9572327044025157, 1.1823819262171171)
(0.9563636363636364, 1.1842971470568495)
CPU times: user 162 ms, sys: 14 ms, total: 176 ms
Wall time: 157 ms


Ez a szöveg mindkét modellnek nehezebb az előzőnél, de az LSTM itt is jelentősen jobban teljesít.

**Egyazon szöveg helyes (vagy kevésbé hibás) és hibás változatának értékei:**

In [99]:
print(string1_correct)
print(string1_ocr)
print("N-gram:")
print("good:", bi_ngram_model.metrics_on_string(string1_correct, combine_function=multiply, padded=True))
print("bad:", bi_ngram_model.metrics_on_string(string1_ocr, combine_function=multiply, padded=True))

print("LSTM:")
print("good:", bilstm_model.metrics_on_string(string1_correct, padded=True))
print("bad:", bilstm_model.metrics_on_string(string1_ocr, padded=True))

A Matáv Adományvonal lehetővé teszi, hogy ön egyszerűen és gyorsan juttasson célba adományt, míg a nonprofit szervezetek pályázati úton, rövid idő alatt juthatnak nagyobb összegekhez egészségmegőrző, szociális es gyógyító programjaikhoz.
A MatPv AdomPny vonal /hl^'etVvö tesiri, hgyy ön egyszerűen és gyorsan UittUssson cáia adományt, míg a non-profit szervezetek pályázati úton, dVvdJ kföalatt uutaatnak nagobür összegekhez ^fS^^e^í^zö, tooPPHs és gyógyító'programjaihhzz.
N-gram:
good: (0.8270042194092827, 1.8078382126904953)
bad: (0.49361702127659574, 373.90907289106514)
LSTM:
good: (0.9620253164556962, 1.1726669829284149)
bad: (0.5574468085106383, 73.51003281684778)


In [100]:
print(string2_good_ocr)
print(string2_bad_ocr)
print("N-gram:")
print("good:", bi_ngram_model.metrics_on_string(string2_good_ocr, combine_function=multiply, padded=True))
print("bad:", bi_ngram_model.metrics_on_string(string2_bad_ocr, combine_function=multiply, padded=True))

print("LSTM:")
print("good:", bilstm_model.metrics_on_string(string2_good_ocr, padded=True))
print("bad:", bilstm_model.metrics_on_string(string2_bad_ocr, padded=True))

Környezetvédelem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól. a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése) 
K^t^r^yt^zz^t^i^tédielem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól. a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése) 
N-gram:
good: (0.8099547511312217, 2.6654477629365356)
bad: (0.7248908296943232, 8.619916800511165)
LSTM:
good: (0.9140271493212669, 1.5980523885190805)
bad: (0.8253275109170306, 5.964297914679948)


In [101]:
print(string3_correct)
print(string3_ocr)
print("N-gram:")
print("good:", bi_ngram_model.metrics_on_string(string3_correct, combine_function=multiply, padded=True))
print("bad:", bi_ngram_model.metrics_on_string(string3_ocr, combine_function=multiply, padded=True))

print("LSTM:")
print("good:", bilstm_model.metrics_on_string(string3_correct, padded=True))
print("bad:", bilstm_model.metrics_on_string(string3_ocr, padded=True))

értékesítő iroda nyitvatartása: hétfő-péntek 8-19h
értíkesítto Iroda m'itfatartdsa: Mjü-ln)Hl<rk 8-19b
N-gram:
good: (0.74, 5.155941483942883)
bad: (0.1568627450980392, 4970.396662142565)
LSTM:
good: (0.86, 2.7747810468382914)
bad: (0.29411764705882354, 622.2232398932931)


Mindkét modell világosan elkülöníti egymástól a hibás és a jó szöveget. A jó szöveg perplexitása és az előrejelzés pontossága mindhárom szöveg esetében látványosan jobb az LSTM-mel számítva, mint az n-gram-modellel. Az, hogy a rossz szöveg perplexitása az n-gram-modell szerint ennyivel rosszabb, mint a jó szövegé, döntően annak a következménye, hogy az n-gram-modellben nem vezettem be a valószínűségekre olyan alsó küszöböt, mint az LSTM-modellnél.

Az kicsit meglepő, hogy a hármas jó szöveget sokkal valószínűtlenebbnek ítéli meg az LSTM-modell, mint a kettes jó szöveget, pedig a hármas emberi szemmel nézve hibátlan, míg a kettes szövegben látványos problémák vannak (*du-* ... *góktól*, kötőjel-szóköz-kötőjel, mondatvégi pont után kisbetű).

**OCR-szemét:**

In [102]:
print(string4)
print(string5)

print("N-gram:")
print(bi_ngram_model.metrics_on_string(string4, combine_function=multiply, padded=True))
print(bi_ngram_model.metrics_on_string(string5, combine_function=multiply, padded=True))

print("LSTM:")
print(bilstm_model.metrics_on_string(string4, padded=True))
print(bilstm_model.metrics_on_string(string5, padded=True))

az Átadva az  to^a KeaeáéNt AaaéWkéFedAa cat (atwNtaaA. aww^y
A a L ét • a  ffc Mkm bML A hWiWWHK1**(re a nMtjjww áHő
N-gram:
(0.18032786885245902, 259906.4372180176)
(0.16363636363636364, 224285.99319624595)
LSTM:
(0.19672131147540983, 2917.6296826774906)
(0.12727272727272726, 2947.0715565223795)


A szemét mindkét modell szerint drasztikusan rosszabb perplexitású, mint bármelyik magyar nyelvű szöveg.

**Nem magyar nyelvű input:**

In [103]:
print(string_en)
print("N-gram:")
print(bi_ngram_model.metrics_on_string(string_en, combine_function=multiply))
print("LSTM:")
print(bilstm_model.metrics_on_string(string_en, print_string=True))

Day Three of PAX East 2022 is done and dusted for another year, which means it's time for another daily round-up video. We had another jam-packed schedule today, checking out Dome Keeper (formerly known as Ludum Dare darling Dome Romantik) over on the Raw Fury booth, before heading over to the PAX Rising Showcase to check out a specially curated selection of upcoming indie games.
N-gram:
(0.44919786096256686, 16.246758630922756)
LSTM:
L Plrt 2000 és Hibpland Sistek formSdapte,nLears Ceits winrotSnas Teto formSdapteraDallygboansson videó. DibMítmSdacte,aRan JarzariSchapen,nBurnl, Csicking jtt Honi Larnen SCocmalroo(row)téz Jotas Wova Walkan, Her, ramantee, Ipersif The Loa Pode Cottta Retorttholring OkeraLo The MPL Zeling Show-orenTorChick jtt " specialee kariner Sereation ofrsrlani
(0.4147727272727273, 9.234163029420852)


A tiszta angol szöveg perplexitása egyik modell szerint sem közelít a magyar szövegekéhez, tehát idegen nyelvű szöveg felismerésére is jól működik mindkét modell. A `print_string` outputján látszik, hogy egyetlen szót sem talál el az LSTM-modell, tehát a tanítókorpuszban valószínűleg nemigen találkozott számottevő angol nyelvű részletekkel.

**Szemétszűrés**:

In [35]:
for line in long_ocr_string.split('\n'):
    if len(line) < 31:
        continue
    print(line)
    print('\t', bilstm_model.metrics_on_string(line, padded=True))

NameError: name 'long_ocr_string' is not defined

Itt lejjebb lehet tenni a perplexitás küszöbét, mint az n-gram-modellnél, mert a meggyőzően jó sorok perplexitása következetesen 3 alatt van. Némi ráhagyással 4-re állítom be.

In [105]:
for line in long_ocr_string.split('\n'):
    if len(line) < 31:
        continue
    perpl = bilstm_model.metrics_on_string(line, padded=True)[1]
    if perpl < 4:
        print(line)
        print('\t', perpl)

Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
	 3.2975684161057406
* A szeptemberi időponthoz további 10% engedmény'
	 2.916731235797478
A Kereskedelmi és Hitelbank Rt letétkezelésében lévő alapok nettó eszközérték Ft (egy jegy re jutó nettó eszközérték) adatai:
	 1.7965669658566392
Mély fájdalommal tudatjuk mindazokkal, akik ismerték és szerették, hogy KÜSTEL RICHÁRD oki. elektromérnök, közgazdász életének 78. évében, hősiesen viselt súlyos betegségben 2002. szeptember 19-én elhunyt.
	 1.6138524278032564
Végső búcsúztatása a Farkasréti temetőben lesz október 10-én 10.30 órakor. A gyászoló család.
	 1.1897200036937081
A szeptember 28-al keresztrejtvény nyertese:
	 1.7432089771265302
Nyereményét, a Nagy Képes Világőrténet, és a Magyarország családjai CD-Romot, postázzuk!
	 2.4815064739063475


In [106]:
for line in long_ocr_string.split('\n'):
    if len(line) < 31:
        continue
    perpl = bilstm_model.metrics_on_string(line, padded=True)[1]
    if perpl > 4:
        print(line)
        print('\t', perpl)

X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
	 232.0532517664282
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
	 58.734038801783896
korutuzas ♦ üdülés Djerba szigetén,
	 12.196808127860377
X nap hottf**** fp.-val	69 900 H/fótól
	 278.6365657051853
botéi***** fp.-val	96 900 Fl/főtől
	 373.91935888994794
Nílusi hajóul közvetlen chalerjarultal. 5* m luxuslupóval: 129 900 Ftíőtől
	 15.329949348407402
TH-: 2M»-O<.*2. 2M-OH33 TH.: .M2-O.MO. .152-WM* MTI. l«Mr«l 44M. nd.
	 718.2612341802306
ajánlaton Portugália .bombaáron* 'épülő, 7 nap teljes p.. X. 14-20.. 21-27 189 900 Ft/« DáFSpanyolorazág-Marokkó. rep. 13 nap
	 16.635379009819914
3/4* száll, fél. ill, teljes p. X. 28-XI 9. 349 900 Ft/« Prága-Kariovy Vary, busz 5 nap fp„ X. 19-23.39 900-47 900 Ft/« München, Sörfesztivál, busz, 3 nap reggeli, IX. 27-29.. X. 4-6 29 900 R/«
	 6.575673132110289
Cseh várak, városok ét kas

Ugyanolyan meggyőzően működik, mint az n-gram-modellel.